# Polarsでデータサイエンス100本ノック（構造化データ加工編）

## ライブラリ準備

今回は `./data` ディレクトリのcsvから直接データを読み込むのでsql関連のライブラリは必要なし。

In [1]:
import os
# 今回はpolarsを用いる
#import pandas as pd
import polars as pl
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from imblearn.under_sampling import RandomUnderSampler

## データ確認

今回用いるデータを確認する。  
元データは [100knocks-preprocess](https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess) の `/docker/doc/data/` 以下からコピーした。  

データを確認しつつデータ型も確認する。  
データ型はER図で確認できるが、日付型のフォーマットなどが気になる。

### 全データ

In [2]:
!ls ./data/

100knocks_ER.png  customer.csv	product.csv  store.csv
category.csv	  geocode.csv	receipt.csv


### customer.csv

In [3]:
!cat ./data/customer.csv | head -n 3

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
cat: write error: Broken pipe


`birth_day` と `application_date` で日付のフォーマットが違う...  
`birth_day` はそのまま日付型で読み込めそうだが `application_date` は一度文字列で読み込んだ後に日付型に変換する。

In [4]:
customer_dtypes = {
    "customer_id": pl.Utf8, # polarsの文字列型
    "customer_name": pl.Utf8,
    "gender_cd": pl.UInt8, # polarsの符号なし整数型
    "gender": pl.Utf8,
    "birth_day": pl.Date, # polarsの日付型
    "age": pl.UInt8,
    "postal_cd": pl.Utf8,
    "address": pl.Utf8,
    "application_store_cd": pl.Utf8,
    "application_date": pl.Utf8, # まずはpolarsの文字列型で読み込んで後で日付型に直す
    "status_cd": pl.Utf8
}

df_customer = pl.read_csv("./data/customer.csv", dtypes=customer_dtypes)
df_customer.head(3)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,u8,str,date,u8,str,str,str,str,str
"""CS021313000114...","""大野 あや子""",1,"""女性""",1981-04-29,37,"""259-1113""","""神奈川県伊勢原市粟窪****...","""S14021""","""20150905""","""0-00000000-0"""
"""CS037613000071...","""六角 雅彦""",9,"""不明""",1952-04-01,66,"""136-0076""","""東京都江東区南砂******...","""S13037""","""20150414""","""0-00000000-0"""
"""CS031415000172...","""宇多田 貴美子""",1,"""女性""",1976-10-04,42,"""151-0053""","""東京都渋谷区代々木*****...","""S13031""","""20150529""","""D-20100325-C"""


In [5]:
# application_dateを日付型に変換し、birth_dayと同じフォーマットにする
# 日付型にすればデフォルトで同じフォーマットになる
df_customer = df_customer.with_columns(
    pl.col("application_date").str.strptime(pl.Date, "%Y%m%d")
)

df_customer.head(3)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,u8,str,date,u8,str,str,str,date,str
"""CS021313000114...","""大野 あや子""",1,"""女性""",1981-04-29,37,"""259-1113""","""神奈川県伊勢原市粟窪****...","""S14021""",2015-09-05,"""0-00000000-0"""
"""CS037613000071...","""六角 雅彦""",9,"""不明""",1952-04-01,66,"""136-0076""","""東京都江東区南砂******...","""S13037""",2015-04-14,"""0-00000000-0"""
"""CS031415000172...","""宇多田 貴美子""",1,"""女性""",1976-10-04,42,"""151-0053""","""東京都渋谷区代々木*****...","""S13031""",2015-05-29,"""D-20100325-C"""


In [6]:
# df_customerのdtypesを修正しておく
customer_dtypes = {k: v for k, v in zip(customer_dtypes.keys(), df_customer.dtypes)}
customer_dtypes

{'customer_id': Utf8,
 'customer_name': Utf8,
 'gender_cd': UInt8,
 'gender': Utf8,
 'birth_day': Date,
 'age': UInt8,
 'postal_cd': Utf8,
 'address': Utf8,
 'application_store_cd': Utf8,
 'application_date': Date,
 'status_cd': Utf8}

データ数の確認

In [7]:
df_customer.shape

(21971, 11)

欠損値の確認

In [8]:
df_customer.null_count()

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0


### product.csv

In [9]:
!cat ./data/product.csv | head -n 3

﻿product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
P040101001,04,0401,040101,198,149
P040101002,04,0401,040101,218,164
cat: write error: Broken pipe


In [10]:
product_dtypes = {
    "product_cd": pl.Utf8,
    "category_major_cd": pl.Utf8,
    "category_medium_cd": pl.Utf8,
    "category_small_cd": pl.Utf8,
    "unit_price": pl.UInt64, # 単価。とりあえず符号なし32bit整数で読み込んでみて、より大きくする必要があれば64bitにする
    "unit_cost": pl.UInt64, # 原価。とりあえず符号なし32bit整数で読み込んでみて、より大きくする必要があれば64bitにする
}

df_product = pl.read_csv("data/product.csv", dtypes=product_dtypes)
# 読み込めたのでprice, costは32bitで十分だったらしい
df_product.head(3)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
str,str,str,str,u64,u64
"""P040101001""","""04""","""0401""","""040101""",198,149
"""P040101002""","""04""","""0401""","""040101""",218,164
"""P040101003""","""04""","""0401""","""040101""",230,173


データ数の確認

In [11]:
df_product.shape

(10030, 6)

欠損値の確認

In [12]:
df_product.null_count()

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
u32,u32,u32,u32,u32,u32
0,0,0,0,7,7


`product.csv` ってマスタだと思うんだがnullあるのか。

### store.csv

In [13]:
!cat ./data/store.csv | head -n 3

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698.0
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0


In [14]:
store_dtypes = {
    "store_cd": pl.Utf8,
    "store_name": pl.Utf8,
    "prefecture_cd": pl.Utf8, # コードなので一応文字列で読み込んでおく
    "prefecture": pl.Utf8,
    "address": pl.Utf8,
    "address_kana": pl.Utf8,
    "tel_no": pl.Utf8,
    "longitude": pl.Float32,
    "latitude": pl.Float32,
    "floor_area": pl.Float32 # 店舗の床面積
}

df_store = pl.read_csv("data/store.csv", dtypes=store_dtypes)
df_store.head(3)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f32,f32,f32
"""S12014""","""千草台店""","""12""","""千葉県""","""千葉県千葉市稲毛区千草台一丁...","""チバケンチバシイナゲクチグサ...","""043-123-4003""",140.117996,35.63559,1698.0
"""S13002""","""国分寺店""","""13""","""東京都""","""東京都国分寺市本多二丁目""","""トウキョウトコクブンジシホン...","""042-123-4008""",139.480194,35.705662,1735.0
"""S14010""","""菊名店""","""14""","""神奈川県""","""神奈川県横浜市港北区菊名一丁...","""カナガワケンヨコハマシコウホ...","""045-123-4032""",139.632599,35.500488,1732.0


データ数の確認

In [15]:
df_store.shape

(53, 10)

欠損値の確認

In [16]:
df_store.null_count()

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0


### category.csv

In [17]:
!cat ./data/category.csv | head -n 3

﻿category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
04,惣菜,0401,御飯類,040101,弁当類
04,惣菜,0401,御飯類,040102,寿司類


In [18]:
category_dtypes = {
    "category_major_cd": pl.Utf8,
    "category_major_name": pl.Utf8,
    "category_medium_cd": pl.Utf8,
    "category_medium_name": pl.Utf8,
    "category_small_cd": pl.Utf8,
    "category_small_name": pl.Utf8
}

df_category = pl.read_csv("data/category.csv", dtypes=category_dtypes)
df_category.head(3)

category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
str,str,str,str,str,str
"""04""","""惣菜""","""0401""","""御飯類""","""040101""","""弁当類"""
"""04""","""惣菜""","""0401""","""御飯類""","""040102""","""寿司類"""
"""04""","""惣菜""","""0402""","""佃煮類""","""040201""","""魚介佃煮類"""


データ数の確認

In [19]:
df_category.shape

(228, 6)

欠損値の確認

In [20]:
df_category.null_count()

category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


### geocode.csv

In [21]:
!cat ./data/geocode.csv | head -n 3

﻿postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
060-0000,北海道,札幌市中央区,,,,北海道札幌市中央区,141.34103,43.05513
064-0941,北海道,札幌市中央区,旭ケ丘,,,北海道札幌市中央区旭ケ丘,141.31972,43.04223
cat: write error: Broken pipe


In [22]:
geocode_dtypes = {
    "postal_cd": pl.Utf8,
    "prefecture": pl.Utf8,
    "city": pl.Utf8,
    "town": pl.Utf8,
    "street": pl.Utf8,
    "address": pl.Utf8,
    "full_address": pl.Utf8,
    "longitude": pl.Float32,
    "latitude": pl.Float32
}

df_geocode = pl.read_csv("data/geocode.csv", dtypes=geocode_dtypes)
df_geocode.head(3)

postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
str,str,str,str,str,str,str,f32,f32
"""060-0000""","""北海道""","""札幌市中央区""",null,null,null,"""北海道札幌市中央区""",141.341034,43.05513
"""064-0941""","""北海道""","""札幌市中央区""","""旭ケ丘""",null,null,"""北海道札幌市中央区旭ケ丘""",141.319717,43.042229
"""060-0042""","""北海道""","""札幌市中央区""","""大通西""",null,"""１丁目""","""北海道札幌市中央区大通西１丁...",141.356369,43.06102


データ数の確認

In [23]:
df_geocode.shape

(127252, 9)

欠損値の確認

In [24]:
df_geocode.null_count()

postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,1954,126117,120244,0,0,0


### receipt.csv

In [25]:
!cat ./data/receipt.csv | head -n 3

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
cat: write error: Broken pipe


`receipt.csv` も日付のフォーマットを `customer.csv` の `birth_date` と合わせる。  
初めに文字列型で読み込んで後で日付型に直す。  

`sales_epoch` はエポック秒を表しているようなので、まずは文字列型で読み込み後で `pl.Datetime` に変換する。 
ER図に記載はないが、Asia/Tokyoのタイムゾーンで読み込んでおく。(UTCじゃないよな...？)

In [26]:
receipt_dtypes = {
    "sales_ymd": pl.Utf8,
    "sales_epoch": pl.Utf8, # 文字列型で読み込み、後でDatetimeに変換
    "store_cd": pl.Utf8,
    "receipt_no": pl.UInt32,
    "receipt_sub_no": pl.UInt8,
    "customer_id": pl.Utf8,
    "product_cd": pl.Utf8,
    "quantity": pl.UInt8,
    "amount": pl.UInt32
}

df_receipt = pl.read_csv("data/receipt.csv", dtypes=receipt_dtypes)
df_receipt.head(3)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
str,str,str,u32,u8,str,str,u8,u32
"""20181103""","""1541203200""","""S14006""",112,1,"""CS006214000001...","""P070305012""",1,158
"""20181118""","""1542499200""","""S13008""",1132,2,"""CS008415000097...","""P070701017""",1,81
"""20170712""","""1499817600""","""S14028""",1102,1,"""CS028414000014...","""P060101005""",1,170


In [27]:
df_receipt = df_receipt.with_columns([
    pl.col("sales_ymd").str.strptime(pl.Date, "%Y%m%d"),
    pl.col("sales_epoch").str.strptime(pl.Datetime, "%s").dt.replace_time_zone("Asia/Tokyo")
])

df_receipt.head(3)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
date,"datetime[μs, Asia/Tokyo]",str,u32,u8,str,str,u8,u32
2018-11-03,2018-11-03 00:00:00 JST,"""S14006""",112,1,"""CS006214000001...","""P070305012""",1,158
2018-11-18,2018-11-18 00:00:00 JST,"""S13008""",1132,2,"""CS008415000097...","""P070701017""",1,81
2017-07-12,2017-07-12 00:00:00 JST,"""S14028""",1102,1,"""CS028414000014...","""P060101005""",1,170


In [28]:
# 型を修正しておく
receipt_dtypes = {k: v for k, v in zip(receipt_dtypes.keys(), df_receipt.dtypes)}
receipt_dtypes

{'sales_ymd': Date,
 'sales_epoch': Datetime(tu='us', tz='Asia/Tokyo'),
 'store_cd': Utf8,
 'receipt_no': UInt32,
 'receipt_sub_no': UInt8,
 'customer_id': Utf8,
 'product_cd': Utf8,
 'quantity': UInt8,
 'amount': UInt32}

データ数の確認

In [29]:
df_receipt.shape

(104681, 9)

欠損値の確認

In [30]:
df_receipt.null_count()

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0


日付のフォーマットずれはあるあるだが...出会うとゲンナリする。日付文字列、ダメ、絶対。  
データの行数も最大で10万行くらいなのでそれほど実行時間はかからなさそう。
`product` に欠損値があることには注意か。  

何となくここまででかなり問題を解いてしまった気もするがここから順番に問題を解いていく。

## P-001
レシート明細データ（df_receipt）から全項⽬の先頭10件を表⽰し、どのようなデータを
保有しているか⽬視で確認せよ。

In [31]:
df_receipt.head(10)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
date,"datetime[μs, Asia/Tokyo]",str,u32,u8,str,str,u8,u32
2018-11-03,2018-11-03 00:00:00 JST,"""S14006""",112,1,"""CS006214000001...","""P070305012""",1,158
2018-11-18,2018-11-18 00:00:00 JST,"""S13008""",1132,2,"""CS008415000097...","""P070701017""",1,81
2017-07-12,2017-07-12 00:00:00 JST,"""S14028""",1102,1,"""CS028414000014...","""P060101005""",1,170
2019-02-05,2019-02-05 00:00:00 JST,"""S14042""",1132,1,"""ZZ000000000000...","""P050301001""",1,25
2018-08-21,2018-08-21 00:00:00 JST,"""S14025""",1102,2,"""CS025415000050...","""P060102007""",1,90
2019-06-05,2019-06-05 00:00:00 JST,"""S13003""",1112,1,"""CS003515000195...","""P050102002""",1,138
2018-12-05,2018-12-05 00:00:00 JST,"""S14024""",1102,2,"""CS024514000042...","""P080101005""",1,30
2019-09-22,2019-09-22 00:00:00 JST,"""S14040""",1102,1,"""CS040415000178...","""P070501004""",1,128
2017-05-04,2017-05-04 00:00:00 JST,"""S13020""",1112,2,"""ZZ000000000000...","""P071302010""",1,770


型も出してくれるの地味にありがたい。

## P-002
レシート明細データ（df_receipt）から売上年⽉⽇（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上⾦額（amount）の順に列を指定し、10件表⽰せよ。

In [32]:
df_receipt.select(["sales_ymd", "customer_id", "product_cd", "amount"]).head(10)

sales_ymd,customer_id,product_cd,amount
date,str,str,u32
2018-11-03,"""CS006214000001...","""P070305012""",158
2018-11-18,"""CS008415000097...","""P070701017""",81
2017-07-12,"""CS028414000014...","""P060101005""",170
2019-02-05,"""ZZ000000000000...","""P050301001""",25
2018-08-21,"""CS025415000050...","""P060102007""",90
2019-06-05,"""CS003515000195...","""P050102002""",138
2018-12-05,"""CS024514000042...","""P080101005""",30
2019-09-22,"""CS040415000178...","""P070501004""",128
2017-05-04,"""ZZ000000000000...","""P071302010""",770


## P-003
レシート明細データ（df_receipt）から売上年⽉⽇（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上⾦額（amount）の順に列を指定し、10件表⽰せよ。ただし、sales_ymdsales_dateに項⽬名を変更しながら抽出すること。

In [33]:
df_receipt.select([
    pl.col("sales_ymd").alias("sales_date"), "customer_id", "product_cd", "amount"
]).head(10)

sales_date,customer_id,product_cd,amount
date,str,str,u32
2018-11-03,"""CS006214000001...","""P070305012""",158
2018-11-18,"""CS008415000097...","""P070701017""",81
2017-07-12,"""CS028414000014...","""P060101005""",170
2019-02-05,"""ZZ000000000000...","""P050301001""",25
2018-08-21,"""CS025415000050...","""P060102007""",90
2019-06-05,"""CS003515000195...","""P050102002""",138
2018-12-05,"""CS024514000042...","""P080101005""",30
2019-09-22,"""CS040415000178...","""P070501004""",128
2017-05-04,"""ZZ000000000000...","""P071302010""",770


`pl.col("sales_ymd")` みたいな指定の仕方は面白い。エクスプレッションといって「処理」そのものを与えているらしい（メソッドってことか？）。  
`pl.col("sales_ymd").alias("sales_date")` で「"sales_ymd"という列を選択して、"sales_date"という名前に変更する」という「処理」を定義している。  
その「処理」をデータフレームに与えることで変形する、という思想らしい。

## P-004
レシート明細データ（df_receipt）から売上⽇（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上⾦額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。

- 顧客ID（customer_id）が"CS018205000001"

In [34]:
df_receipt.filter(
    pl.col("customer_id") == "CS018205000001"
).select(
    ["sales_ymd", "customer_id", "product_cd", "amount"]
)

sales_ymd,customer_id,product_cd,amount
date,str,str,u32
2018-09-11,"""CS018205000001...","""P071401012""",2200
2018-04-14,"""CS018205000001...","""P060104007""",600
2017-06-14,"""CS018205000001...","""P050206001""",990
2017-06-14,"""CS018205000001...","""P060702015""",108
2019-02-16,"""CS018205000001...","""P071005024""",102
2018-04-14,"""CS018205000001...","""P071101002""",278
2019-02-26,"""CS018205000001...","""P070902035""",168
2019-09-24,"""CS018205000001...","""P060805001""",495
2019-02-26,"""CS018205000001...","""P071401020""",2200


## P-005
レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
- 顧客ID（customer_id）が"CS018205000001"
- 売上金額（amount）が1,000以上

In [35]:
df_receipt.filter(
    (pl.col("customer_id") == "CS018205000001") & (pl.col("amount") >= 1000)
).select(
    ["sales_ymd", "customer_id", "product_cd", "amount"]
)

sales_ymd,customer_id,product_cd,amount
date,str,str,u32
2018-09-11,"""CS018205000001...","""P071401012""",2200
2019-02-26,"""CS018205000001...","""P071401020""",2200
2018-09-11,"""CS018205000001...","""P071401005""",1100


## P-006
レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
- 顧客ID（customer_id）が"CS018205000001"
- 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [36]:
df_receipt.select(
    ["sales_ymd", "customer_id", "product_cd", "quantity", "amount"]
).filter(
    (pl.col("customer_id") == "CS018205000001") &
    ((pl.col("amount") >= 1000) | (pl.col("quantity") >= 5))
)

sales_ymd,customer_id,product_cd,quantity,amount
date,str,str,u8,u32
2018-09-11,"""CS018205000001...","""P071401012""",1,2200
2018-04-14,"""CS018205000001...","""P060104007""",6,600
2017-06-14,"""CS018205000001...","""P050206001""",5,990
2019-02-26,"""CS018205000001...","""P071401020""",1,2200
2018-09-11,"""CS018205000001...","""P071401005""",1,1100


## P-007
レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
- 顧客ID（customer_id）が"CS018205000001"
- 売上金額（amount）が1,000以上2,000以下

In [37]:
df_receipt.select(
    ["sales_ymd", "customer_id", "product_cd", "amount"]
).filter(
    (pl.col("customer_id") == "CS018205000001") &
    (pl.col("amount").is_between(1000, 2000))
)

sales_ymd,customer_id,product_cd,amount
date,str,str,u32
2018-09-11,"""CS018205000001...","""P071401005""",1100


In [38]:
# closed="both"がデフォルトらしい
df_receipt.select(
    ["sales_ymd", "customer_id", "product_cd", "amount"]
).filter(
    (pl.col("customer_id") == "CS018205000001") &
    (pl.col("amount").is_between(1000, 2000, closed="both"))
)

sales_ymd,customer_id,product_cd,amount
date,str,str,u32
2018-09-11,"""CS018205000001...","""P071401005""",1100


## P-008
レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
- 顧客ID（customer_id）が"CS018205000001"
- 商品コード（product_cd）が"P071401019"以外

In [39]:
df_receipt.select(
    ["sales_ymd", "customer_id", "product_cd", "amount"]
).filter(
    (pl.col("customer_id") == "CS018205000001") &
    (pl.col("product_cd") != "P071401019")
)

sales_ymd,customer_id,product_cd,amount
date,str,str,u32
2018-09-11,"""CS018205000001...","""P071401012""",2200
2018-04-14,"""CS018205000001...","""P060104007""",600
2017-06-14,"""CS018205000001...","""P050206001""",990
2017-06-14,"""CS018205000001...","""P060702015""",108
2019-02-16,"""CS018205000001...","""P071005024""",102
2018-04-14,"""CS018205000001...","""P071101002""",278
2019-02-26,"""CS018205000001...","""P070902035""",168
2019-09-24,"""CS018205000001...","""P060805001""",495
2019-02-26,"""CS018205000001...","""P071401020""",2200


## P-009
以下の処理において、出力結果を変えずにORをANDに書き換えよ。  
`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [40]:
# とりあえず問題文のコードの出力を確認
#df_store.query('not(prefecture_cd == "13" | floor_area > 900)')

# エラー出た。polarsにqueryはないっぽい

In [41]:
df_store.filter(
    (pl.col("prefecture_cd") != "13") &
    (pl.col("floor_area") <= 900)
)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f32,f32,f32
"""S14046""","""北山田店""","""14""","""神奈川県""","""神奈川県横浜市都筑区北山田一...","""カナガワケンヨコハマシツヅキ...","""045-123-4049""",139.591599,35.56189,831.0
"""S14011""","""日吉本町店""","""14""","""神奈川県""","""神奈川県横浜市港北区日吉本町...","""カナガワケンヨコハマシコウホ...","""045-123-4033""",139.631607,35.546551,890.0
"""S12013""","""習志野店""","""12""","""千葉県""","""千葉県習志野市芝園一丁目""","""チバケンナラシノシシバゾノイ...","""047-123-4002""",140.022003,35.661221,808.0


## P-010
店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [42]:
df_store.filter(
    pl.col("store_cd").str.starts_with("S14")
).head(10)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f32,f32,f32
"""S14010""","""菊名店""","""14""","""神奈川県""","""神奈川県横浜市港北区菊名一丁...","""カナガワケンヨコハマシコウホ...","""045-123-4032""",139.632599,35.500488,1732.0
"""S14033""","""阿久和店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西...","""カナガワケンヨコハマシセヤク...","""045-123-4043""",139.496094,35.459179,1495.0
"""S14036""","""相模原中央店""","""14""","""神奈川県""","""神奈川県相模原市中央二丁目""","""カナガワケンサガミハラシチュ...","""042-123-4045""",139.371597,35.573269,1679.0
"""S14040""","""長津田店""","""14""","""神奈川県""","""神奈川県横浜市緑区長津田みな...","""カナガワケンヨコハマシミドリ...","""045-123-4046""",139.499405,35.523979,1548.0
"""S14050""","""阿久和西店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西...","""カナガワケンヨコハマシセヤク...","""045-123-4053""",139.496094,35.459179,1830.0
"""S14028""","""二ツ橋店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区二ツ橋町...","""カナガワケンヨコハマシセヤク...","""045-123-4042""",139.496307,35.463039,1574.0
"""S14012""","""本牧和田店""","""14""","""神奈川県""","""神奈川県横浜市中区本牧和田""","""カナガワケンヨコハマシナカク...","""045-123-4034""",139.658203,35.421558,1341.0
"""S14046""","""北山田店""","""14""","""神奈川県""","""神奈川県横浜市都筑区北山田一...","""カナガワケンヨコハマシツヅキ...","""045-123-4049""",139.591599,35.56189,831.0
"""S14022""","""逗子店""","""14""","""神奈川県""","""神奈川県逗子市逗子一丁目""","""カナガワケンズシシズシイッチ...","""046-123-4036""",139.578903,35.296421,1838.0


## P-011
顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [43]:
df_customer.filter(
    pl.col("customer_id").str.ends_with("1")
).head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,u8,str,date,u8,str,str,str,date,str
"""CS037613000071...","""六角 雅彦""",9,"""不明""",1952-04-01,66,"""136-0076""","""東京都江東区南砂******...","""S13037""",2015-04-14,"""0-00000000-0"""
"""CS028811000001...","""堀井 かおり""",1,"""女性""",1933-03-27,86,"""245-0016""","""神奈川県横浜市泉区和泉町**...","""S14028""",2016-01-15,"""0-00000000-0"""
"""CS040412000191...","""川井 郁恵""",1,"""女性""",1977-01-05,42,"""226-0021""","""神奈川県横浜市緑区北八朔町*...","""S14040""",2015-11-01,"""1-20091025-4"""
"""CS028314000011...","""小菅 あおい""",1,"""女性""",1983-11-26,35,"""246-0038""","""神奈川県横浜市瀬谷区宮沢**...","""S14028""",2015-11-23,"""1-20080426-5"""
"""CS039212000051...","""藤島 恵梨香""",1,"""女性""",1997-02-03,22,"""166-0001""","""東京都杉並区阿佐谷北****...","""S13039""",2017-11-21,"""1-20100215-4"""
"""CS015412000111...","""松居 奈月""",1,"""女性""",1972-10-04,46,"""136-0071""","""東京都江東区亀戸******...","""S13015""",2015-06-29,"""0-00000000-0"""
"""CS004702000041...","""野島 洋""",0,"""男性""",1943-08-24,75,"""176-0022""","""東京都練馬区向山******...","""S13004""",2017-02-18,"""0-00000000-0"""
"""CS041515000001...","""栗田 千夏""",1,"""女性""",1967-01-02,52,"""206-0001""","""東京都多摩市和田******...","""S13041""",2016-04-22,"""E-20100803-F"""
"""CS029313000221...","""北条 ひかり""",1,"""女性""",1987-06-19,31,"""279-0011""","""千葉県浦安市美浜******...","""S12029""",2018-08-10,"""0-00000000-0"""


## P-012
店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [44]:
df_store.filter(
    pl.col("address").str.contains("横浜市")
)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f32,f32,f32
"""S14010""","""菊名店""","""14""","""神奈川県""","""神奈川県横浜市港北区菊名一丁...","""カナガワケンヨコハマシコウホ...","""045-123-4032""",139.632599,35.500488,1732.0
"""S14033""","""阿久和店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西...","""カナガワケンヨコハマシセヤク...","""045-123-4043""",139.496094,35.459179,1495.0
"""S14040""","""長津田店""","""14""","""神奈川県""","""神奈川県横浜市緑区長津田みな...","""カナガワケンヨコハマシミドリ...","""045-123-4046""",139.499405,35.523979,1548.0
"""S14050""","""阿久和西店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西...","""カナガワケンヨコハマシセヤク...","""045-123-4053""",139.496094,35.459179,1830.0
"""S14028""","""二ツ橋店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区二ツ橋町...","""カナガワケンヨコハマシセヤク...","""045-123-4042""",139.496307,35.463039,1574.0
"""S14012""","""本牧和田店""","""14""","""神奈川県""","""神奈川県横浜市中区本牧和田""","""カナガワケンヨコハマシナカク...","""045-123-4034""",139.658203,35.421558,1341.0
"""S14046""","""北山田店""","""14""","""神奈川県""","""神奈川県横浜市都筑区北山田一...","""カナガワケンヨコハマシツヅキ...","""045-123-4049""",139.591599,35.56189,831.0
"""S14011""","""日吉本町店""","""14""","""神奈川県""","""神奈川県横浜市港北区日吉本町...","""カナガワケンヨコハマシコウホ...","""045-123-4033""",139.631607,35.546551,890.0
"""S14048""","""中川中央店""","""14""","""神奈川県""","""神奈川県横浜市都筑区中川中央...","""カナガワケンヨコハマシツヅキ...","""045-123-4051""",139.575806,35.549122,1657.0


## P-013
顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [45]:
df_customer.filter(
    pl.col("status_cd").str.contains(r"^[A-F]")
).head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,u8,str,date,u8,str,str,str,date,str
"""CS031415000172...","""宇多田 貴美子""",1,"""女性""",1976-10-04,42,"""151-0053""","""東京都渋谷区代々木*****...","""S13031""",2015-05-29,"""D-20100325-C"""
"""CS015414000103...","""奥野 陽子""",1,"""女性""",1977-08-09,41,"""136-0073""","""東京都江東区北砂******...","""S13015""",2015-07-22,"""B-20100609-B"""
"""CS011215000048...","""芦田 沙耶""",1,"""女性""",1992-02-01,27,"""223-0062""","""神奈川県横浜市港北区日吉本町...","""S14011""",2015-02-28,"""C-20100421-9"""
"""CS029415000023...","""梅田 里穂""",1,"""女性""",1976-01-17,43,"""279-0043""","""千葉県浦安市富士見*****...","""S12029""",2015-06-10,"""D-20100918-E"""
"""CS035415000029...","""寺沢 真希""",9,"""不明""",1977-09-27,41,"""158-0096""","""東京都世田谷区玉川台****...","""S13035""",2014-12-20,"""F-20101029-F"""
"""CS031415000106...","""宇野 由美子""",1,"""女性""",1970-02-26,49,"""151-0053""","""東京都渋谷区代々木*****...","""S13031""",2015-02-01,"""F-20100511-E"""
"""CS029215000025...","""石倉 美帆""",1,"""女性""",1993-09-28,25,"""279-0022""","""千葉県浦安市今川******...","""S12029""",2015-07-08,"""B-20100820-C"""
"""CS033605000005...","""猪股 雄太""",0,"""男性""",1955-12-05,63,"""246-0031""","""神奈川県横浜市瀬谷区瀬谷**...","""S14033""",2015-04-25,"""F-20100917-E"""
"""CS033415000229...","""板垣 菜々美""",1,"""女性""",1977-11-07,41,"""246-0021""","""神奈川県横浜市瀬谷区二ツ橋町...","""S14033""",2015-07-12,"""F-20100326-E"""


## P-014
顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [46]:
df_customer.filter(
    pl.col("status_cd").str.contains(r"[1-9]$")
).head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,u8,str,date,u8,str,str,str,date,str
"""CS001215000145...","""田崎 美紀""",1,"""女性""",1995-03-29,24,"""144-0055""","""東京都大田区仲六郷*****...","""S13001""",2017-06-05,"""6-20090929-2"""
"""CS033513000180...","""安斎 遥""",1,"""女性""",1962-07-11,56,"""241-0823""","""神奈川県横浜市旭区善部町**...","""S14033""",2015-07-28,"""6-20080506-5"""
"""CS011215000048...","""芦田 沙耶""",1,"""女性""",1992-02-01,27,"""223-0062""","""神奈川県横浜市港北区日吉本町...","""S14011""",2015-02-28,"""C-20100421-9"""
"""CS040412000191...","""川井 郁恵""",1,"""女性""",1977-01-05,42,"""226-0021""","""神奈川県横浜市緑区北八朔町*...","""S14040""",2015-11-01,"""1-20091025-4"""
"""CS009315000023...","""皆川 文世""",1,"""女性""",1980-04-15,38,"""154-0012""","""東京都世田谷区駒沢*****...","""S13009""",2015-03-19,"""5-20080322-1"""
"""CS015315000033...","""福士 璃奈子""",1,"""女性""",1983-03-17,36,"""135-0043""","""東京都江東区塩浜******...","""S13015""",2014-10-24,"""4-20080219-3"""
"""CS023513000066...","""神戸 そら""",1,"""女性""",1961-12-17,57,"""210-0005""","""神奈川県川崎市川崎区東田町*...","""S14023""",2015-09-15,"""5-20100524-9"""
"""CS035513000134...","""市川 美帆""",1,"""女性""",1960-03-27,59,"""156-0053""","""東京都世田谷区桜******...","""S13035""",2015-02-27,"""8-20100711-9"""
"""CS001515000263...","""高松 夏空""",1,"""女性""",1962-11-09,56,"""144-0051""","""東京都大田区西蒲田*****...","""S13001""",2016-08-12,"""1-20100804-1"""


## P-015
顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [47]:
df_customer.filter(
    pl.col("status_cd").str.contains("^[A-F].*[1-9]$")
).head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,u8,str,date,u8,str,str,str,date,str
"""CS011215000048...","""芦田 沙耶""",1,"""女性""",1992-02-01,27,"""223-0062""","""神奈川県横浜市港北区日吉本町...","""S14011""",2015-02-28,"""C-20100421-9"""
"""CS022513000105...","""島村 貴美子""",1,"""女性""",1962-03-12,57,"""249-0002""","""神奈川県逗子市山の根****...","""S14022""",2015-03-20,"""A-20091115-7"""
"""CS001515000096...","""水野 陽子""",9,"""不明""",1960-11-29,58,"""144-0053""","""東京都大田区蒲田本町****...","""S13001""",2015-06-14,"""A-20100724-7"""
"""CS013615000053...","""西脇 季衣""",1,"""女性""",1953-10-18,65,"""261-0026""","""千葉県千葉市美浜区幕張西**...","""S12013""",2015-01-28,"""B-20100329-6"""
"""CS020412000161...","""小宮 薫""",1,"""女性""",1974-05-21,44,"""174-0042""","""東京都板橋区東坂下*****...","""S13020""",2015-08-22,"""B-20081021-3"""
"""CS001215000097...","""竹中 あさみ""",1,"""女性""",1990-07-25,28,"""146-0095""","""東京都大田区多摩川*****...","""S13001""",2017-03-15,"""A-20100211-2"""
"""CS035212000007...","""内村 恵梨香""",1,"""女性""",1990-12-04,28,"""152-0023""","""東京都目黒区八雲******...","""S13035""",2015-10-13,"""B-20101018-6"""
"""CS002515000386...","""野田 コウ""",1,"""女性""",1963-05-30,55,"""185-0013""","""東京都国分寺市西恋ケ窪***...","""S13002""",2016-04-10,"""C-20100127-8"""
"""CS001615000372...","""稲垣 寿々花""",1,"""女性""",1956-10-29,62,"""144-0035""","""東京都大田区南蒲田*****...","""S13001""",2017-04-03,"""A-20100104-1"""


## P-016
店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [48]:
df_store.filter(
    pl.col("tel_no").str.contains(r"^[0-9]{3}-[0-9]{3}-[0-9]{4}$")
)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f32,f32,f32
"""S12014""","""千草台店""","""12""","""千葉県""","""千葉県千葉市稲毛区千草台一丁...","""チバケンチバシイナゲクチグサ...","""043-123-4003""",140.117996,35.63559,1698.0
"""S13002""","""国分寺店""","""13""","""東京都""","""東京都国分寺市本多二丁目""","""トウキョウトコクブンジシホン...","""042-123-4008""",139.480194,35.705662,1735.0
"""S14010""","""菊名店""","""14""","""神奈川県""","""神奈川県横浜市港北区菊名一丁...","""カナガワケンヨコハマシコウホ...","""045-123-4032""",139.632599,35.500488,1732.0
"""S14033""","""阿久和店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西...","""カナガワケンヨコハマシセヤク...","""045-123-4043""",139.496094,35.459179,1495.0
"""S14036""","""相模原中央店""","""14""","""神奈川県""","""神奈川県相模原市中央二丁目""","""カナガワケンサガミハラシチュ...","""042-123-4045""",139.371597,35.573269,1679.0
"""S14040""","""長津田店""","""14""","""神奈川県""","""神奈川県横浜市緑区長津田みな...","""カナガワケンヨコハマシミドリ...","""045-123-4046""",139.499405,35.523979,1548.0
"""S14050""","""阿久和西店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西...","""カナガワケンヨコハマシセヤク...","""045-123-4053""",139.496094,35.459179,1830.0
"""S13052""","""森野店""","""13""","""東京都""","""東京都町田市森野三丁目""","""トウキョウトマチダシモリノサ...","""042-123-4030""",139.438293,35.552929,1087.0
"""S14028""","""二ツ橋店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区二ツ橋町...","""カナガワケンヨコハマシセヤク...","""045-123-4042""",139.496307,35.463039,1574.0


## P-017
顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [49]:
df_customer.sort("birth_day").head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,u8,str,date,u8,str,str,str,date,str
"""CS003813000014...","""村山 菜々美""",1,"""女性""",1928-11-26,90,"""182-0007""","""東京都調布市菊野台*****...","""S13003""",2016-02-14,"""0-00000000-0"""
"""CS026813000004...","""吉村 朝陽""",1,"""女性""",1928-12-14,90,"""251-0043""","""神奈川県藤沢市辻堂元町***...","""S14026""",2015-07-23,"""0-00000000-0"""
"""CS018811000003...","""熊沢 美里""",1,"""女性""",1929-01-07,90,"""204-0004""","""東京都清瀬市野塩******...","""S13018""",2015-04-03,"""0-00000000-0"""
"""CS027803000004...","""内村 拓郎""",0,"""男性""",1929-01-12,90,"""251-0031""","""神奈川県藤沢市鵠沼藤が谷**...","""S14027""",2015-12-27,"""0-00000000-0"""
"""CS013801000003...","""天野 拓郎""",0,"""男性""",1929-01-15,90,"""274-0824""","""千葉県船橋市前原東*****...","""S12013""",2016-01-20,"""0-00000000-0"""
"""CS001814000022...","""鶴田 里穂""",1,"""女性""",1929-01-28,90,"""144-0045""","""東京都大田区南六郷*****...","""S13001""",2016-10-12,"""A-20090415-7"""
"""CS016815000002...","""山元 美紀""",1,"""女性""",1929-02-22,90,"""184-0005""","""東京都小金井市桜町*****...","""S13016""",2015-06-29,"""C-20090923-C"""
"""CS009815000003...","""中田 里穂""",1,"""女性""",1929-04-08,89,"""154-0014""","""東京都世田谷区新町*****...","""S13009""",2015-04-21,"""D-20091021-E"""
"""CS012813000013...","""宇野 南朋""",1,"""女性""",1929-04-09,89,"""231-0806""","""神奈川県横浜市中区本牧町**...","""S14012""",2015-07-12,"""0-00000000-0"""


## P-018
顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [50]:
df_customer.sort("birth_day", reverse=True).head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,u8,str,date,u8,str,str,str,date,str
"""CS035114000004...","""大村 美里""",1,"""女性""",2007-11-25,11,"""156-0053""","""東京都世田谷区桜******...","""S13035""",2015-06-19,"""6-20091205-6"""
"""CS022103000002...","""福山 はじめ""",9,"""不明""",2007-10-02,11,"""249-0006""","""神奈川県逗子市逗子*****...","""S14022""",2016-09-09,"""0-00000000-0"""
"""CS002113000009...","""柴田 真悠子""",1,"""女性""",2007-09-17,11,"""184-0014""","""東京都小金井市貫井南町***...","""S13002""",2016-03-04,"""0-00000000-0"""
"""CS004115000014...","""松井 京子""",1,"""女性""",2007-08-09,11,"""165-0031""","""東京都中野区上鷺宮*****...","""S13004""",2016-11-20,"""1-20081231-1"""
"""CS002114000010...","""山内 遥""",1,"""女性""",2007-06-03,11,"""184-0015""","""東京都小金井市貫井北町***...","""S13002""",2016-09-20,"""6-20100510-1"""
"""CS025115000002...","""小柳 夏希""",1,"""女性""",2007-04-18,11,"""245-0018""","""神奈川県横浜市泉区上飯田町*...","""S14025""",2016-01-16,"""D-20100913-D"""
"""CS002113000025...","""広末 まなみ""",1,"""女性""",2007-03-30,12,"""184-0015""","""東京都小金井市貫井北町***...","""S13002""",2017-10-30,"""0-00000000-0"""
"""CS033112000003...","""長野 美紀""",1,"""女性""",2007-03-22,12,"""245-0051""","""神奈川県横浜市戸塚区名瀬町*...","""S14033""",2015-06-06,"""0-00000000-0"""
"""CS007115000006...","""福岡 瞬""",1,"""女性""",2007-03-10,12,"""285-0845""","""千葉県佐倉市西志津*****...","""S12007""",2015-11-18,"""F-20101016-F"""


## P-019
 レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [51]:
df_receipt.select([
    pl.col("customer_id"),
    pl.col("amount"),
    pl.col("amount").rank("dense", reverse=True).alias("rank")
]).sort("rank").head(10)

customer_id,amount,rank
str,u32,u32
"""CS011415000006...",10925,1
"""ZZ000000000000...",6800,2
"""CS028605000002...",5780,3
"""CS015515000034...",5480,4
"""ZZ000000000000...",5480,4
"""ZZ000000000000...",5480,4
"""ZZ000000000000...",5440,5
"""CS021515000089...",5440,5
"""CS015515000083...",5280,6


## P-020
レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [52]:
df_receipt.select([
    pl.col("customer_id"),
    pl.col("amount"),
    pl.col("amount").rank("ordinal", reverse=True).alias("rank")
]).sort("rank").head(10)

customer_id,amount,rank
str,u32,u32
"""CS011415000006...",10925,1
"""ZZ000000000000...",6800,2
"""CS028605000002...",5780,3
"""CS015515000034...",5480,4
"""ZZ000000000000...",5480,5
"""ZZ000000000000...",5480,6
"""ZZ000000000000...",5440,7
"""CS021515000089...",5440,8
"""CS015515000083...",5280,9


## P-021
レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [53]:
df_receipt.height

104681

## P-022
レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [54]:
df_receipt.select(pl.n_unique("customer_id"))

customer_id
u32
8307


## P-023
レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [55]:
df_receipt.groupby("store_cd").agg([
    pl.sum("amount").suffix("_sum"),
    pl.sum("quantity").suffix("_sum")
])

store_cd,amount_sum,quantity_sum
str,u32,i64
"""S14028""",786145,2458
"""S12007""",638761,2099
"""S13032""",790501,2491
"""S13038""",708884,2337
"""S14022""",651328,2047
"""S14027""",714550,2303
"""S13004""",779373,2390
"""S13019""",827833,2541
"""S14045""",458484,1398


## P-024
レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [56]:
df_receipt.groupby("customer_id").agg(
    pl.max("sales_ymd")
).head(10)

customer_id,sales_ymd
str,date
"""CS035515000170...",2019-10-18
"""CS031414000008...",2019-10-07
"""CS014615000013...",2019-01-31
"""CS001415000474...",2018-03-11
"""CS034515000164...",2019-09-05
"""CS008415000271...",2019-03-09
"""CS029415000080...",2019-02-25
"""CS024815000012...",2018-04-15
"""CS023803000002...",2018-12-27


## P-025
レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [57]:
df_receipt.groupby("customer_id").agg(
    pl.min("sales_ymd")
).head(10)

customer_id,sales_ymd
str,date
"""CS004413000039...",2019-10-17
"""CS003414000017...",2017-02-02
"""CS005514000148...",2018-09-07
"""CS016615000116...",2017-07-29
"""CS026513000072...",2017-01-27
"""CS037511000055...",2019-07-26
"""CS031413000017...",2017-02-10
"""CS018513000149...",2018-05-11
"""CS029415000222...",2017-01-16


## P-026
レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [58]:
df_receipt.groupby("customer_id").agg([
    pl.max("sales_ymd").suffix("_max"),
    pl.min("sales_ymd").suffix("_min")
]).filter(
    pl.col("sales_ymd_max") != pl.col("sales_ymd_min")
).head(10)

customer_id,sales_ymd_max,sales_ymd_min
str,date,date
"""CS030415000034...",2019-10-20,2017-02-05
"""CS001411000014...",2017-12-06,2017-08-28
"""CS008414000063...",2019-08-25,2017-01-26
"""CS037215000046...",2019-09-14,2017-03-16
"""CS013515000134...",2019-08-31,2017-07-04
"""CS031415000019...",2019-09-09,2017-07-16
"""CS011415000149...",2019-04-24,2017-05-08
"""CS027411000001...",2019-02-14,2017-07-05
"""CS004314000055...",2019-04-23,2018-10-16


## P-027
レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [59]:
df_receipt.groupby("store_cd").agg(
    pl.mean("amount").suffix("_mean")
).sort(
    "amount_mean", reverse=True
).head(5)

store_cd,amount_mean
str,f64
"""S13052""",402.86747
"""S13015""",351.11196
"""S13003""",350.915519
"""S14010""",348.791262
"""S13001""",348.470386


## P-028
レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [60]:
df_receipt.groupby("store_cd").agg(
    pl.median("amount").suffix("_median")
).sort(
    "amount_median", reverse=True
).head(5)

store_cd,amount_median
str,f64
"""S13052""",190.0
"""S14010""",188.0
"""S14050""",185.0
"""S13003""",180.0
"""S13018""",180.0


## P-029
レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [61]:
df_receipt.groupby("store_cd").agg(
    pl.col("product_cd").mode().alias("product_cd_mode")
).select([
    pl.col("store_cd"),
    pl.col("product_cd_mode").arr.first()
]).head(10)

store_cd,product_cd_mode
str,str
"""S14040""","""P060303001"""
"""S13035""","""P040503001"""
"""S13037""","""P060303001"""
"""S14045""","""P060303001"""
"""S13004""","""P060303001"""
"""S13019""","""P071401001"""
"""S14034""","""P060303001"""
"""S13016""","""P071102001"""
"""S13020""","""P071401001"""


## P-030
レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [62]:
df_receipt.groupby("store_cd").agg(
    pl.var("amount", ddof=1).suffix("_var") # ddof=1はデフォルト
).sort(
    "amount_var", reverse=True
).head(5)

store_cd,amount_var
str,f64
"""S13052""",441863.252526
"""S14011""",306442.242432
"""S14034""",297068.39274
"""S13001""",295558.842618
"""S13015""",295427.197086


## P-031
レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [63]:
df_receipt.groupby("store_cd").agg(
    pl.std("amount", ddof=1).suffix("_std") # ddof=1はデフォルト
).sort("amount_std", reverse=True).head(10)

store_cd,amount_std
str,f64
"""S13052""",664.727954
"""S14011""",553.572256
"""S14034""",545.039808
"""S13001""",543.653237
"""S13015""",543.532149
"""S13035""",527.372229
"""S14045""",516.374585
"""S14046""",514.257363
"""S14010""",508.496179


## P-032
レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [64]:
df_receipt.select([
    pl.min("amount").suffix("_min"),
    pl.quantile("amount", 0.25).suffix("_q25"),
    pl.quantile("amount", 0.5).suffix("_q50"),
    pl.quantile("amount", 0.75).suffix("_q75"),
    pl.max("amount").suffix("_max")
])

amount_min,amount_q25,amount_q50,amount_q75,amount_max
u32,f64,f64,f64,u32
10,102.0,170.0,288.0,10925


## P-033
レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [65]:
df_receipt.groupby("store_cd").agg(
    pl.mean("amount").suffix("_mean")
).filter(
    pl.col("amount_mean") >= 330
)

store_cd,amount_mean
str,f64
"""S12013""",330.19413
"""S14010""",348.791262
"""S14047""",330.077073
"""S13001""",348.470386
"""S14026""",332.340588
"""S13004""",330.943949
"""S14045""",330.082073
"""S13052""",402.86747
"""S13015""",351.11196


## P-034
レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [66]:
df_receipt.filter(
    ~pl.col("customer_id").str.starts_with("Z")
).groupby("customer_id").agg(
    pl.sum("amount").suffix("_mean")
).select(
    pl.col("amount_mean")
).mean()

amount_mean
f64
2547.742235


## P-035
レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [67]:
df_receipt.filter(
    ~pl.col("customer_id").str.starts_with("Z")
).groupby("customer_id").agg(
    pl.sum("amount").suffix("_mean")
).with_columns(
    pl.mean("amount_mean").alias("all_mean")
).filter(
    pl.col("amount_mean") >= pl.col("all_mean")
).select(["customer_id", "amount_mean"]).head(10)

customer_id,amount_mean
str,u32
"""CS025515000161...",6339
"""CS002614000180...",3597
"""CS030415000034...",15468
"""CS018515000016...",2995
"""CS009415000112...",3397
"""CS038414000013...",7499
"""CS042515000008...",6485
"""CS029515000251...",4754
"""CS026415000175...",8876


## P-036
レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [68]:
df_receipt.join(
    df_store.select(["store_cd", "store_name"]),
    on="store_cd",
    how="inner", # innerがデフォルト
).head(10)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
date,"datetime[μs, Asia/Tokyo]",str,u32,u8,str,str,u8,u32,str
2018-11-03,2018-11-03 00:00:00 JST,"""S14006""",112,1,"""CS006214000001...","""P070305012""",1,158,"""葛が谷店"""
2018-11-18,2018-11-18 00:00:00 JST,"""S13008""",1132,2,"""CS008415000097...","""P070701017""",1,81,"""成城店"""
2017-07-12,2017-07-12 00:00:00 JST,"""S14028""",1102,1,"""CS028414000014...","""P060101005""",1,170,"""二ツ橋店"""
2019-02-05,2019-02-05 00:00:00 JST,"""S14042""",1132,1,"""ZZ000000000000...","""P050301001""",1,25,"""新山下店"""
2018-08-21,2018-08-21 00:00:00 JST,"""S14025""",1102,2,"""CS025415000050...","""P060102007""",1,90,"""大和店"""
2019-06-05,2019-06-05 00:00:00 JST,"""S13003""",1112,1,"""CS003515000195...","""P050102002""",1,138,"""狛江店"""
2018-12-05,2018-12-05 00:00:00 JST,"""S14024""",1102,2,"""CS024514000042...","""P080101005""",1,30,"""三田店"""
2019-09-22,2019-09-22 00:00:00 JST,"""S14040""",1102,1,"""CS040415000178...","""P070501004""",1,128,"""長津田店"""
2017-05-04,2017-05-04 00:00:00 JST,"""S13020""",1112,2,"""ZZ000000000000...","""P071302010""",1,770,"""十条仲原店"""


## P-037
商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [69]:
df_product.join(
    df_category.select([
        "category_small_cd", "category_small_name" # category_small_cdがPK
    ]),
    on="category_small_cd",
    how="inner"
).head(10)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
str,str,str,str,u64,u64,str
"""P040101001""","""04""","""0401""","""040101""",198,149,"""弁当類"""
"""P040101002""","""04""","""0401""","""040101""",218,164,"""弁当類"""
"""P040101003""","""04""","""0401""","""040101""",230,173,"""弁当類"""
"""P040101004""","""04""","""0401""","""040101""",248,186,"""弁当類"""
"""P040101005""","""04""","""0401""","""040101""",268,201,"""弁当類"""
"""P040101006""","""04""","""0401""","""040101""",298,224,"""弁当類"""
"""P040101007""","""04""","""0401""","""040101""",338,254,"""弁当類"""
"""P040101008""","""04""","""0401""","""040101""",420,315,"""弁当類"""
"""P040101009""","""04""","""0401""","""040101""",498,374,"""弁当類"""


## P-038
顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [70]:
df_customer.filter(
    (pl.col("gender_cd") == 1) &
    (~pl.col("customer_id").str.starts_with("Z"))
).select([
    "customer_id",
]).join(
    df_receipt.select(["customer_id", "amount"]),
    on="customer_id",
    how="left"
).groupby("customer_id").agg(
    pl.sum("amount").fill_null(0)
).head(10)

customer_id,amount
str,u32
"""CS004515000149...",654
"""CS008413000181...",0
"""CS037315000119...",578
"""CS002312000283...",0
"""CS026315000060...",0
"""CS003515000290...",4938
"""CS011215000043...",5769
"""CS029512000031...",0
"""CS025415000208...",3609


## P-039
レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [71]:
df_member = df_receipt.filter(
    ~pl.col("customer_id").str.starts_with("Z")
)

df_frequency_top20 = df_member.groupby("customer_id").agg(
    pl.col("sales_ymd").n_unique().alias("frequency_ymd")
).sort(
    "frequency_ymd", reverse=True
).head(20)

df_monetary_top20 = df_member.groupby("customer_id").agg(
    pl.sum("amount").suffix("_sum")
).sort(
    "amount_sum", reverse=True
).head(20)

df_outer_frequency_monetary_top20 = df_frequency_top20.join(
    df_monetary_top20,
    on="customer_id",
    how="outer"
)

df_outer_frequency_monetary_top20.select("customer_id")

customer_id
str
"""CS017415000097..."
"""CS015415000185..."
"""CS031414000051..."
"""CS028415000007..."
"""CS001605000009..."
"""CS010214000010..."
"""CS016415000141..."
"""CS006515000023..."
"""CS011414000106..."


## P-040
全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [72]:
df_store.select(
    "store_cd"
).join(
    df_product.select("product_cd"),
    how="cross"
).height

531590

## P-041
レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [73]:
df_receipt.groupby("sales_ymd").agg(
    pl.sum("amount").suffix("_sum")
).sort(
    "sales_ymd"
).with_columns(
    (pl.col("amount_sum") - pl.col("amount_sum").shift()).alias("amount_diff")
).head(10)

sales_ymd,amount_sum,amount_diff
date,u32,u32
2017-01-01,33723,null
2017-01-02,24165,4294957738
2017-01-03,27503,3338
2017-01-04,36165,8662
2017-01-05,37830,1665
2017-01-06,32387,4294961853
2017-01-07,23415,4294958324
2017-01-08,24737,1322
2017-01-09,26718,1981


謎

In [74]:
df_receipt.groupby("sales_ymd").agg(
    pl.sum("amount").cast(pl.Int64).suffix("_sum")
).sort(
    "sales_ymd"
).with_columns(
    (pl.col("amount_sum") - pl.col("amount_sum").shift()).alias("amount_diff")
).head(10)

sales_ymd,amount_sum,amount_diff
date,i64,i64
2017-01-01,33723,null
2017-01-02,24165,-9558
2017-01-03,27503,3338
2017-01-04,36165,8662
2017-01-05,37830,1665
2017-01-06,32387,-5443
2017-01-07,23415,-8972
2017-01-08,24737,1322
2017-01-09,26718,1981


なるほど、castする必要があるのか。

## P-042
レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [75]:
df_receipt.groupby("sales_ymd").agg(
    pl.sum("amount").suffix("_sum")
).with_columns([
    pl.col("amount_sum").shift().alias("amount_sum_lag1"),
    pl.col("amount_sum").shift(2).alias("amount_sum_lag2"),
    pl.col("amount_sum").shift(3).alias("amount_sum_lag3")
]).head(10)

sales_ymd,amount_sum,amount_sum_lag1,amount_sum_lag2,amount_sum_lag3
date,u32,u32,u32,u32
2019-01-24,44468,null,null,null
2018-01-05,31213,44468,null,null
2017-08-30,33864,31213,44468,null
2018-09-18,18850,33864,31213,44468
2017-04-24,35412,18850,33864,31213
2019-10-07,36160,35412,18850,33864
2018-05-13,32856,36160,35412,18850
2019-06-01,46784,32856,36160,35412
2017-06-19,28640,46784,32856,36160


## P-043
レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。  
性別コードは0が男性、1が女性、9が不明を表すものとする。  
ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [76]:
df_pivot = df_receipt.join(
    df_customer,
    on="customer_id",
    how="inner"
).with_columns(
    pl.when(pl.col("age").is_between(0, 10, "left")).then("[0, 10)")
      .when(pl.col("age").is_between(10, 20, "left")).then("[10, 20)")
      .when(pl.col("age").is_between(20, 30, "left")).then("[20, 30)")
      .when(pl.col("age").is_between(30, 40, "left")).then("[30, 40)")
      .when(pl.col("age").is_between(40, 50, "left")).then("[40, 50)")
      .when(pl.col("age").is_between(50, 60, "left")).then("[50, 60)")
      .when(pl.col("age").is_between(60, 70, "left")).then("[60, 70)")
      .when(pl.col("age").is_between(60, 70, "left")).then("[60, 70)")
      .otherwise("[70 )")
      .alias("age_bin")
).pivot(
    "amount",
    "age_bin",
    "gender_cd",
    "sum"
).sort("age_bin").with_columns([
    pl.col("0").alias("men_amount_sum"),
    pl.col("1").alias("women_amount_sum"),
    pl.col("9").alias("unknown_gender_amount_sum")
]).select([
    "age_bin", "men_amount_sum", "women_amount_sum", "unknown_gender_amount_sum"
])

df_pivot

age_bin,men_amount_sum,women_amount_sum,unknown_gender_amount_sum
str,u32,u32,u32
"""[10, 20)""",1591,149836,4317
"""[20, 30)""",72940,1363724,44328
"""[30, 40)""",177322,693047,50441
"""[40, 50)""",19355,9320791,483512
"""[50, 60)""",54320,6685192,342923
"""[60, 70)""",272469,987741,71418
"""[70 )""",59795,298947,7538


## P-044
043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。  
このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [77]:
df_pivot.melt(
    id_vars="age_bin",
    value_vars=["men_amount_sum", "women_amount_sum", "unknown_gender_amount_sum"]
).with_columns([
    pl.when(pl.col("variable") == "men_amount_sum").then("00")
      .when(pl.col("variable") == "women_amount_sum").then("01")
      .when(pl.col("variable") == "unknown_gender_amount_sum").then("99")
      .alias("gender_cd"),
    pl.col("value").alias("amount_sum")
]).select([
    "age_bin", "gender_cd", "amount_sum"
]).sort("age_bin")

age_bin,gender_cd,amount_sum
str,str,u32
"""[10, 20)""","""00""",1591
"""[10, 20)""","""01""",149836
"""[10, 20)""","""99""",4317
"""[20, 30)""","""00""",72940
"""[20, 30)""","""01""",1363724
"""[20, 30)""","""99""",44328
"""[30, 40)""","""00""",177322
"""[30, 40)""","""01""",693047
"""[30, 40)""","""99""",50441


## P-045
顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [78]:
df_customer.select("birth_day").head(10)

birth_day
date
1981-04-29
1952-04-01
1976-10-04
1933-03-27
1995-03-29
1974-09-15
1977-08-09
1973-08-17
1931-05-02


In [79]:
df_customer.with_columns(
    pl.col("birth_day").dt.strftime("%Y%m%d")
).select(["customer_id", "birth_day"]).head(10)

customer_id,birth_day
str,str
"""CS021313000114...","""19810429"""
"""CS037613000071...","""19520401"""
"""CS031415000172...","""19761004"""
"""CS028811000001...","""19330327"""
"""CS001215000145...","""19950329"""
"""CS020401000016...","""19740915"""
"""CS015414000103...","""19770809"""
"""CS029403000008...","""19730817"""
"""CS015804000004...","""19310502"""


## P-046
顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

最初のデータチェックでやってしまっていた...  
一応コードはこちら

```Python
df_customer = df_customer.with_columns(
    pl.col("application_date").str.strptime(pl.Date, "%Y%m%d")
)
```

In [80]:
df_customer.select(["customer_id", "application_date"]).head(10)

customer_id,application_date
str,date
"""CS021313000114...",2015-09-05
"""CS037613000071...",2015-04-14
"""CS031415000172...",2015-05-29
"""CS028811000001...",2016-01-15
"""CS001215000145...",2017-06-05
"""CS020401000016...",2015-02-25
"""CS015414000103...",2015-07-22
"""CS029403000008...",2015-05-15
"""CS015804000004...",2015-06-07


## P-047
レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

こちらもデータチェックの段階ですでにやってしまっていた。

```Python
df_receipt = df_receipt.with_columns([
    pl.col("sales_ymd").str.strptime(pl.Date, "%Y%m%d"),
    pl.col("sales_epoch").str.strptime(pl.Datetime, "%s").dt.replace_time_zone("Asia/Tokyo")
])
```

In [81]:
df_receipt.select(["receipt_no", "receipt_sub_no", "sales_ymd"]).head(10)

receipt_no,receipt_sub_no,sales_ymd
u32,u8,date
112,1,2018-11-03
1132,2,2018-11-18
1102,1,2017-07-12
1132,1,2019-02-05
1102,2,2018-08-21
1112,1,2019-06-05
1102,2,2018-12-05
1102,1,2019-09-22
1112,2,2017-05-04


## P-048
レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

上でdatetime形にするのはやっているので、date形に変換するところだけやる。

In [82]:
df_receipt.with_columns(
    pl.col("sales_epoch").dt.strftime("%Y%m%d").str.strptime(pl.Date, "%Y%m%d").alias("sales_epoch_ymd")
).select(
    ["receipt_no", "receipt_sub_no", "sales_epoch_ymd"]
).head(10)

receipt_no,receipt_sub_no,sales_epoch_ymd
u32,u8,date
112,1,2018-11-03
1132,2,2018-11-18
1102,1,2017-07-12
1132,1,2019-02-05
1102,2,2018-08-21
1112,1,2019-06-05
1102,2,2018-12-05
1102,1,2019-09-22
1112,2,2017-05-04


## P-049
レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [83]:
df_receipt.with_columns(
    pl.col("sales_epoch").dt.year().alias("sales_epoch_year")
).select(
    ["receipt_no", "receipt_sub_no", "sales_epoch_year"]
).head(10)

receipt_no,receipt_sub_no,sales_epoch_year
u32,u8,i32
112,1,2018
1132,2,2018
1102,1,2017
1132,1,2019
1102,2,2018
1112,1,2019
1102,2,2018
1102,1,2019
1112,2,2017


## P-050
レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [84]:
df_receipt.with_columns(
    pl.col("sales_epoch").dt.month().cast(pl.Utf8).str.zfill(2).alias("sales_epoch_month")
).select(
    ["receipt_no", "receipt_sub_no", "sales_epoch_month"]
).head(10)

receipt_no,receipt_sub_no,sales_epoch_month
u32,u8,str
112,1,"""11"""
1132,2,"""11"""
1102,1,"""07"""
1132,1,"""02"""
1102,2,"""08"""
1112,1,"""06"""
1102,2,"""12"""
1102,1,"""09"""
1112,2,"""05"""


## P-051
レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [85]:
df_receipt.with_columns(
    pl.col("sales_epoch").dt.day().cast(pl.Utf8).str.zfill(2).alias("sales_epoch_day")
).select(
    ["receipt_no", "receipt_sub_no", "sales_epoch_day"]
).head(10)

receipt_no,receipt_sub_no,sales_epoch_day
u32,u8,str
112,1,"""03"""
1132,2,"""18"""
1102,1,"""12"""
1132,1,"""05"""
1102,2,"""21"""
1112,1,"""05"""
1102,2,"""05"""
1102,1,"""22"""
1112,2,"""04"""


## P-052
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [86]:
df_receipt.filter(
    ~pl.col("customer_id").str.starts_with("Z")
).groupby("customer_id").agg(
    pl.sum("amount").suffix("_sum")
).select([
    "customer_id",
    "amount_sum",
    pl.when(pl.col("amount_sum") <= 2000).then(0)
      .otherwise(1).alias("amount_bin")
]).head(10)

customer_id,amount_sum,amount_bin
str,u32,i32
"""CS023414000011...",7251,1
"""CS017515000183...",1283,0
"""CS026415000231...",4987,1
"""CS013413000022...",213,0
"""CS024815000009...",1160,0
"""CS037412000058...",255,0
"""CS001415000265...",3524,1
"""CS020414000115...",6794,1
"""CS004514000140...",850,0


## P-053
顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [87]:
df_receipt.select(pl.min("amount"))

amount
u32
10


amountの最小値は10なのでdf_receiptが紐づかないcustomer_idが売り上げ実績のない顧客として扱ってよさそう。

In [88]:
df_customer.with_columns(
    pl.when(pl.col("postal_cd").str.slice(0, 3)
                               .cast(pl.UInt16).is_between(100, 209)).then(1)
      .otherwise(0).alias("is_Tokyo_customer")
).join(
    df_receipt,
    how="left",
    on="customer_id"
).filter(
    pl.col("amount").is_not_null()
).groupby("is_Tokyo_customer").agg(
    pl.col("customer_id").n_unique()
)

is_Tokyo_customer,customer_id
i32,u32
0,3906
1,4400


## P-054
顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [89]:
df_customer.with_columns(
    pl.when(pl.col("address").str.contains("埼玉県")).then(11)
      .when(pl.col("address").str.contains("千葉県")).then(12)
      .when(pl.col("address").str.contains("東京都")).then(13)
      .when(pl.col("address").str.contains("神奈川県")).then(14)
      .alias("prefecture_cd")
).select([
    "customer_id", "address", "prefecture_cd"
]).head(10)

customer_id,address,prefecture_cd
str,str,i32
"""CS021313000114...","""神奈川県伊勢原市粟窪****...",14
"""CS037613000071...","""東京都江東区南砂******...",13
"""CS031415000172...","""東京都渋谷区代々木*****...",13
"""CS028811000001...","""神奈川県横浜市泉区和泉町**...",14
"""CS001215000145...","""東京都大田区仲六郷*****...",13
"""CS020401000016...","""東京都板橋区若木******...",13
"""CS015414000103...","""東京都江東区北砂******...",13
"""CS029403000008...","""千葉県浦安市海楽******...",12
"""CS015804000004...","""東京都江東区北砂******...",13


## P-055
レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。

- 最小値以上第1四分位未満 ・・・ 1を付与
- 第1四分位以上第2四分位未満 ・・・ 2を付与
- 第2四分位以上第3四分位未満 ・・・ 3を付与
- 第3四分位以上 ・・・ 4を付与

In [90]:
df_receipt.groupby("customer_id").agg(
    pl.sum("amount").suffix("_sum")
).with_columns(
    pl.when(pl.col("amount_sum") < pl.quantile("amount_sum", 0.25)).then(1)
      .when((pl.col("amount_sum") >= pl.quantile("amount_sum", 0.25)) &
            (pl.col("amount_sum") < pl.quantile("amount_sum", 0.5))).then(2)
      .when((pl.col("amount_sum") >= pl.quantile("amount_sum", 0.5)) &
            (pl.col("amount_sum") < pl.quantile("amount_sum", 0.75))).then(3)
      .otherwise(4)
      .alias("amount_bin")
).select([
    "customer_id", "amount_sum", "amount_bin"
]).head(10)

customer_id,amount_sum,amount_bin
str,u32,i32
"""CS013512000083...",496,1
"""CS003515000007...",3657,4
"""CS046415000005...",4429,4
"""CS019515000171...",6075,4
"""CS030315000116...",423,1
"""CS025513000179...",2394,3
"""CS001515000429...",415,1
"""CS007615000013...",4128,4
"""CS035513000149...",2608,3


## P-056
顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。  
ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [91]:
df_customer.with_columns(
    pl.when(pl.col("age") < 10).then("10歳未満")
      .when((pl.col("age") >= 10) & (pl.col("age") < 20)).then("10代")
      .when((pl.col("age") >= 20) & (pl.col("age") < 30)).then("20代")
      .when((pl.col("age") >= 30) & (pl.col("age") < 40)).then("30代")
      .when((pl.col("age") >= 40) & (pl.col("age") < 50)).then("40代")
      .when((pl.col("age") >= 50) & (pl.col("age") < 60)).then("50代")
      .otherwise("60代")
      .alias("年代")
).select([
    "customer_id", "birth_day", "年代"
]).head(10)

customer_id,birth_day,年代
str,date,str
"""CS021313000114...",1981-04-29,"""30代"""
"""CS037613000071...",1952-04-01,"""60代"""
"""CS031415000172...",1976-10-04,"""40代"""
"""CS028811000001...",1933-03-27,"""60代"""
"""CS001215000145...",1995-03-29,"""20代"""
"""CS020401000016...",1974-09-15,"""40代"""
"""CS015414000103...",1977-08-09,"""40代"""
"""CS029403000008...",1973-08-17,"""40代"""
"""CS015804000004...",1931-05-02,"""60代"""


## P-057
056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。  
組み合わせを表すカテゴリの値は任意とする。

In [92]:
df_customer.with_columns(
    pl.when(pl.col("age") < 10).then("10歳未満")
      .when((pl.col("age") >= 10) & (pl.col("age") < 20)).then("10代")
      .when((pl.col("age") >= 20) & (pl.col("age") < 30)).then("20代")
      .when((pl.col("age") >= 30) & (pl.col("age") < 40)).then("30代")
      .when((pl.col("age") >= 40) & (pl.col("age") < 50)).then("40代")
      .when((pl.col("age") >= 50) & (pl.col("age") < 60)).then("50代")
      .otherwise("60代")
      .alias("era")
).select([
    "customer_id", "birth_day",
    pl.concat_str([pl.col("era"), pl.col("gender_cd")])
]).head(10)

customer_id,birth_day,era
str,date,str
"""CS021313000114...",1981-04-29,"""30代1"""
"""CS037613000071...",1952-04-01,"""60代9"""
"""CS031415000172...",1976-10-04,"""40代1"""
"""CS028811000001...",1933-03-27,"""60代1"""
"""CS001215000145...",1995-03-29,"""20代1"""
"""CS020401000016...",1974-09-15,"""40代0"""
"""CS015414000103...",1977-08-09,"""40代1"""
"""CS029403000008...",1973-08-17,"""40代0"""
"""CS015804000004...",1931-05-02,"""60代0"""


## P-058
顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

ChatGPT様に教えてもらった。  
確かに `*` でアンパックすれば行けるわ。

In [93]:
df_customer.select([
    pl.col("customer_id"),
    *pl.get_dummies(df_customer.select("gender_cd"))
]).head(10)

customer_id,gender_cd_0,gender_cd_1,gender_cd_9
str,u8,u8,u8
"""CS021313000114...",0,1,0
"""CS037613000071...",0,0,1
"""CS031415000172...",0,1,0
"""CS028811000001...",0,1,0
"""CS001215000145...",0,1,0
"""CS020401000016...",1,0,0
"""CS015414000103...",0,1,0
"""CS029403000008...",1,0,0
"""CS015804000004...",1,0,0


## P-059
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。  
標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。  
ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [94]:
df_receipt_customer_amount = df_receipt.filter(
    ~pl.col("customer_id").str.starts_with("Z")
).groupby(
    "customer_id"
).agg(
    pl.sum("amount").suffix("_sum")
)

mean_val = df_receipt_customer_amount.select("amount_sum").mean()
std_val = df_receipt_customer_amount.select("amount_sum").std()

# 標準化
df_receipt_customer_amount.with_columns([
    pl.col("customer_id"),
    pl.col("amount_sum"),
    ((pl.col("amount_sum") - mean_val) / std_val).alias("standardized_amount_sum")
]).head(10)

customer_id,amount_sum,standardized_amount_sum
str,u32,f64
"""CS037512000123...",416,-0.783535
"""CS014614000056...",70,-0.910709
"""CS011415000219...",2484,-0.023429
"""CS029514000009...",3032,0.177992
"""CS022215000021...",348,-0.808528
"""CS037412000091...",115,-0.894169
"""CS033511000026...",1028,-0.55859
"""CS040314000065...",243,-0.847122
"""CS032214000027...",1313,-0.453837


## P-060
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。  
ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [95]:
df_receipt_customer_amount = df_receipt.filter(
    ~pl.col("customer_id").str.starts_with("Z")
).groupby(
    "customer_id"
).agg(
    pl.sum("amount").suffix("_sum")
)

min_val = df_receipt_customer_amount.select("amount_sum").min()
max_val = df_receipt_customer_amount.select("amount_sum").max()

# 正規化
df_receipt_customer_amount.with_columns([
    pl.col("customer_id"),
    pl.col("amount_sum"),
    ((pl.col("amount_sum") - min_val) / (max_val - min_val)).alias("normalized_amount_sum")
]).head(10)

customer_id,amount_sum,normalized_amount_sum
str,u32,f64
"""CS021514000082...",5544,0.237835
"""CS039513000072...",3492,0.148679
"""CS009413000147...",466,0.017205
"""CS023414000132...",1006,0.040667
"""CS037205000004...",4045,0.172706
"""CS032415000225...",3363,0.143074
"""CS027211000005...",989,0.039929
"""CS014515000067...",4417,0.188869
"""CS010412000025...",426,0.015468


## P-061
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。  
ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [96]:
df_receipt.filter(
    ~pl.col("customer_id").str.starts_with("Z")
).groupby("customer_id").agg(
    pl.sum("amount").suffix("_sum")
).with_columns(
    pl.col("amount_sum").log(base=10).suffix("_log10")
).head(10)

customer_id,amount_sum,amount_sum_log10
str,u32,f64
"""CS039415000129...",7781,3.891035
"""CS006211000012...",1436,3.157154
"""CS030512000026...",947,2.97635
"""CS011415000253...",5158,3.712481
"""CS003314000140...",372,2.570543
"""CS002515000596...",1466,3.166134
"""CS021513000206...",233,2.367356
"""CS004215000084...",1514,3.180126
"""CS020414000084...",3944,3.595937


## P-062
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。  
ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [97]:
df_receipt.filter(
    ~pl.col("customer_id").str.starts_with("Z")
).groupby("customer_id").agg(
    pl.sum("amount").suffix("_sum")
).with_columns(
    pl.col("amount_sum").log().suffix("_loge")  # logのデフォルトが底e
).head(10)

customer_id,amount_sum,amount_sum_loge
str,u32,f64
"""CS035415000099...",6189,8.730529
"""CS009415000236...",6342,8.754949
"""CS009415000230...",6865,8.834191
"""CS039315000112...",1268,7.145196
"""CS010412000032...",915,6.818924
"""CS002415000155...",1619,7.389564
"""CS030512000166...",1000,6.907755
"""CS035414000054...",5182,8.552946
"""CS012512000069...",1852,7.524021


## P-063
商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [98]:
df_product.select([
    "unit_price", "unit_cost",
    (pl.col("unit_price") - pl.col("unit_cost")).alias("unit_profit")
]).head(10)

unit_price,unit_cost,unit_profit
u64,u64,u64
198,149,49
218,164,54
230,173,57
248,186,62
268,201,67
298,224,74
338,254,84
420,315,105
498,374,124


## P-064
商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。  
ただし、単価と原価には欠損が生じていることに注意せよ。

In [99]:
df_product.select(
    ((pl.col("unit_price") - pl.col("unit_cost")) / pl.col("unit_price")).alias("unit_profit_ratio")
).mean()

unit_profit_ratio
f64
0.249114


## P-065
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。  
ただし、1円未満は切り捨てること。  
そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。  
ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [100]:
df_product.select([
    pl.col("unit_price").alias("unit_price_original"),
    pl.col("unit_cost"),
    (pl.col("unit_cost") * 10 / 7.0).floor().alias("unit_price_new")
]).with_columns(
    ((pl.col("unit_price_new") - pl.col("unit_cost")) / pl.col("unit_price_new")).alias("unit_profit_new")
).head(10)

unit_price_original,unit_cost,unit_price_new,unit_profit_new
u64,u64,f64,f64
198,149,212.0,0.29717
218,164,234.0,0.299145
230,173,247.0,0.299595
248,186,265.0,0.298113
268,201,287.0,0.299652
298,224,320.0,0.3
338,254,362.0,0.298343
420,315,450.0,0.3
498,374,534.0,0.299625


## P-066
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。  
今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。  
そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。  
ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [101]:
df_product.select([
    pl.col("unit_price").alias("unit_price_original"),
    pl.col("unit_cost"),
    (pl.col("unit_cost") * 10 / 7.0).round(0).alias("unit_price_new")
]).with_columns(
    ((pl.col("unit_price_new") - pl.col("unit_cost")) / pl.col("unit_price_new")).alias("unit_profit_new")
).head(10)

unit_price_original,unit_cost,unit_price_new,unit_profit_new
u64,u64,f64,f64
198,149,213.0,0.300469
218,164,234.0,0.299145
230,173,247.0,0.299595
248,186,266.0,0.300752
268,201,287.0,0.299652
298,224,320.0,0.3
338,254,363.0,0.300275
420,315,450.0,0.3
498,374,534.0,0.299625


## P-067
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ  
。今回は、1円未満を切り上げること。  
そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。  
ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [102]:
df_product.select([
    pl.col("unit_price").alias("unit_price_original"),
    pl.col("unit_cost"),
    (pl.col("unit_cost") * 10 / 7.0).ceil().alias("unit_price_new")
]).with_columns(
    ((pl.col("unit_price_new") - pl.col("unit_cost")) / pl.col("unit_price_new")).alias("unit_profit_new")
).head(10)

unit_price_original,unit_cost,unit_price_new,unit_profit_new
u64,u64,f64,f64
198,149,213.0,0.300469
218,164,235.0,0.302128
230,173,248.0,0.302419
248,186,266.0,0.300752
268,201,288.0,0.302083
298,224,320.0,0.3
338,254,363.0,0.300275
420,315,450.0,0.3
498,374,535.0,0.300935


## P-068
商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。  
1円未満の端数は切り捨てとし、結果を10件表示せよ。  
ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [103]:
df_product.select([
    pl.col("unit_price").alias("unit_price_original"),
    (pl.col("unit_price") * 1.1).floor().alias("unit_price_new")
]).head(10)

unit_price_original,unit_price_new
u64,f64
198,217.0
218,239.0
230,253.0
248,272.0
268,294.0
298,327.0
338,371.0
420,462.0
498,547.0


## P-069
レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。  
抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [104]:
df_receipt.join(
    df_product, how="left", on="product_cd"
).with_columns(  # 瓶詰缶詰購入ログにフラグを付ける
    pl.when(pl.col("category_major_cd") == "07").then(1)
      .otherwise(0)
      .alias("is_bought_bottled")
).with_columns(  # 瓶詰缶詰ログのみの購入金額を抽出
    (pl.col("is_bought_bottled") * pl.col("amount")).alias("amount_bottled")
).groupby("customer_id").agg([
    pl.sum("amount").suffix("_sum"),
    pl.sum("amount_bottled").suffix("_sum")
]).select([
    "customer_id", "amount_sum", "amount_bottled_sum",
    (pl.col("amount_bottled_sum") / pl.col("amount_sum")).alias("bottled_ratio")
]).filter(
    pl.col("amount_bottled_sum") != 0
).head(10)

customer_id,amount_sum,amount_bottled_sum,bottled_ratio
str,u32,i64,f64
"""CS012515000007...",2340,607,0.259402
"""CS043415000010...",6707,4785,0.713434
"""CS021411000060...",396,190,0.479798
"""CS001311000059...",2302,102,0.044309
"""CS028412000111...",818,818,1.0
"""CS005615000004...",1498,1100,0.734312
"""CS006415000019...",986,408,0.413793
"""CS024515000022...",2776,2104,0.757925
"""CS032414000107...",6924,3757,0.542605


## P-070
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [105]:
# データチェックの段階ですべて日付型に直しているので引き算するだけ
df_receipt.join(
    df_customer, how="inner", on="customer_id"
).select([
    "customer_id", "sales_ymd", "application_date",
    (pl.col("sales_ymd") - pl.col("application_date")).alias("days_from_application")
]).head(10)

customer_id,sales_ymd,application_date,days_from_application
str,date,date,duration[ms]
"""CS006214000001...",2018-11-03,2015-02-01,1371d
"""CS008415000097...",2018-11-18,2015-03-22,1337d
"""CS028414000014...",2017-07-12,2015-07-11,732d
"""CS025415000050...",2018-08-21,2016-01-31,933d
"""CS003515000195...",2019-06-05,2015-03-06,1552d
"""CS024514000042...",2018-12-05,2015-10-10,1152d
"""CS040415000178...",2019-09-22,2015-06-27,1548d
"""CS027514000015...",2019-10-10,2015-11-01,1439d
"""CS025415000134...",2019-09-18,2015-07-20,1521d


## P-071
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [106]:
df_receipt.join(
    df_customer, how="inner", on="customer_id"
).select([
    "customer_id", "sales_ymd", "application_date",
    ((pl.col("sales_ymd").dt.year() - pl.col("application_date").dt.year()) * 12 +
     pl.col("sales_ymd").dt.month() - pl.col("application_date").dt.month()).alias("month_from_application")
]).head(10)

customer_id,sales_ymd,application_date,month_from_application
str,date,date,i64
"""CS006214000001...",2018-11-03,2015-02-01,45
"""CS008415000097...",2018-11-18,2015-03-22,44
"""CS028414000014...",2017-07-12,2015-07-11,24
"""CS025415000050...",2018-08-21,2016-01-31,31
"""CS003515000195...",2019-06-05,2015-03-06,51
"""CS024514000042...",2018-12-05,2015-10-10,38
"""CS040415000178...",2019-09-22,2015-06-27,51
"""CS027514000015...",2019-10-10,2015-11-01,47
"""CS025415000134...",2019-09-18,2015-07-20,50


## P-072
レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [107]:
df_receipt.join(
    df_customer, how="inner", on="customer_id"
).select([
    "customer_id", "sales_ymd", "application_date",
    (pl.col("sales_ymd").dt.year() - pl.col("application_date").dt.year()).alias("years_from_application")
]).head(10)

customer_id,sales_ymd,application_date,years_from_application
str,date,date,i32
"""CS006214000001...",2018-11-03,2015-02-01,3
"""CS008415000097...",2018-11-18,2015-03-22,3
"""CS028414000014...",2017-07-12,2015-07-11,2
"""CS025415000050...",2018-08-21,2016-01-31,2
"""CS003515000195...",2019-06-05,2015-03-06,4
"""CS024514000042...",2018-12-05,2015-10-10,3
"""CS040415000178...",2019-09-22,2015-06-27,4
"""CS027514000015...",2019-10-10,2015-11-01,4
"""CS025415000134...",2019-09-18,2015-07-20,4


## P-073
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [108]:
df_receipt.join(
    df_customer, how="inner", on="customer_id"
).select([
    "customer_id", "sales_ymd", "application_date",
    (pl.col("sales_ymd").dt.epoch("s") - pl.col("application_date").dt.epoch("s")).alias("epoch_seconds_from_application")
]).head(10)

customer_id,sales_ymd,application_date,epoch_seconds_from_application
str,date,date,i64
"""CS006214000001...",2018-11-03,2015-02-01,118454400
"""CS008415000097...",2018-11-18,2015-03-22,115516800
"""CS028414000014...",2017-07-12,2015-07-11,63244800
"""CS025415000050...",2018-08-21,2016-01-31,80611200
"""CS003515000195...",2019-06-05,2015-03-06,134092800
"""CS024514000042...",2018-12-05,2015-10-10,99532800
"""CS040415000178...",2019-09-22,2015-06-27,133747200
"""CS027514000015...",2019-10-10,2015-11-01,124329600
"""CS025415000134...",2019-09-18,2015-07-20,131414400


## P-074
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [109]:
df_receipt.select([
    "sales_ymd",
    pl.col("sales_ymd").dt.truncate("1w").alias("date_monday"),
    (pl.col("sales_ymd") - pl.col("sales_ymd").dt.truncate("1w")).alias("days_from_monday")
]).head(10)

sales_ymd,date_monday,days_from_monday
date,date,duration[ms]
2018-11-03,2018-10-29,5d
2018-11-18,2018-11-12,6d
2017-07-12,2017-07-10,2d
2019-02-05,2019-02-04,1d
2018-08-21,2018-08-20,1d
2019-06-05,2019-06-03,2d
2018-12-05,2018-12-03,2d
2019-09-22,2019-09-16,6d
2017-05-04,2017-05-01,3d


## P-075
顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [110]:
df_customer.sample(frac=0.1).head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,u8,str,date,u8,str,str,str,date,str
"""CS011513000235...","""安 愛梨""",1,"""女性""",1965-09-19,53,"""223-0064""","""神奈川県横浜市港北区下田町*...","""S14011""",2015-06-21,"""3-20091004-3"""
"""CS040303000002...","""三上 祐一""",0,"""男性""",1981-07-27,37,"""226-0027""","""神奈川県横浜市緑区長津田**...","""S14040""",2015-04-25,"""0-00000000-0"""
"""CS031415000172...","""宇多田 貴美子""",1,"""女性""",1976-10-04,42,"""151-0053""","""東京都渋谷区代々木*****...","""S13031""",2015-05-29,"""D-20100325-C"""
"""CS027613000050...","""西川 千佳子""",1,"""女性""",1953-11-07,65,"""251-0052""","""神奈川県藤沢市藤沢*****...","""S14027""",2015-09-02,"""2-20091102-5"""
"""CS025712000027...","""村山 めぐみ""",1,"""女性""",1945-07-04,73,"""242-0024""","""神奈川県大和市福田*****...","""S14025""",2015-11-29,"""0-00000000-0"""
"""CS001412000541...","""河原 夏希""",1,"""女性""",1976-06-11,42,"""210-0844""","""神奈川県川崎市川崎区渡田新町...","""S13001""",2017-05-03,"""0-00000000-0"""
"""CS021815000012...","""前田 ヒカル""",1,"""女性""",1933-01-10,86,"""259-1133""","""神奈川県伊勢原市東大竹***...","""S14021""",2015-07-28,"""0-00000000-0"""
"""CS029403000008...","""釈 人志""",0,"""男性""",1973-08-17,45,"""279-0003""","""千葉県浦安市海楽******...","""S12029""",2015-05-15,"""0-00000000-0"""
"""CS005512000107...","""田辺 恵麻""",1,"""女性""",1961-01-18,58,"""166-0004""","""東京都杉並区阿佐谷南****...","""S13005""",2015-06-26,"""0-00000000-0"""


## P-076
顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [111]:
# groupbyにsampleを適用することで層化抽出が実現できる
df_customer.groupby("gender_cd").apply(
    lambda x: x.sample(frac=0.1)
).select(
    pl.col("gender_cd").value_counts()
)

gender_cd
struct[2]
"{0,298}"
"{9,107}"
"{1,1791}"


## P-077
レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。  
なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。  
結果は10件表示せよ。

In [112]:
# 顧客単位で売り上げの和を取ったdfを作っておく
df_amount_sum = df_receipt.groupby("customer_id").agg([
    pl.sum("amount"),
    pl.sum("amount").log().suffix("_log")
])

# 平均と標準偏差を求めておく
mean_std = df_amount_sum.select([
    pl.mean("amount_log").alias("mean"),
    pl.std("amount_log").alias("std")
])

# 外れ値のデータを抽出する
df_amount_sum.filter(
    (pl.col("amount_log") > mean_std["mean"] + 3 * mean_std["std"]) |
    (pl.col("amount_log") < mean_std["mean"] - 3 * mean_std["std"])
).select([
    "customer_id", "amount"
]).head(10)

customer_id,amount
str,u32
"""ZZ000000000000...",12395003


## P-078
レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。  
ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。  
なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [113]:
# 顧客単位で売り上げの和を取ったdfを作っておく
df_amount_sum = df_receipt.filter(
    ~pl.col("customer_id").str.starts_with("Z")
).groupby("customer_id").agg(
    pl.sum("amount")
)

# 四分位数を求める
qt = df_amount_sum.select([
    pl.quantile("amount", 0.25).suffix("_q25"),
    pl.quantile("amount", 0.75).suffix("_q75"),
]).with_columns(
    (pl.col("amount_q75") - pl.col("amount_q25")).alias("IQR")
)

# 外れ値データを抽出する
df_amount_sum.filter(
    (pl.col("amount") > qt["amount_q75"] + 1.5 * qt["IQR"]) |
    (pl.col("amount") < qt["amount_q25"] - 1.5 * qt["IQR"])
).head(10)

customer_id,amount
str,u32
"""CS038415000147...",13324
"""CS028414000049...",9386
"""CS001605000009...",18925
"""CS032414000062...",9568
"""CS021515000056...",12036
"""CS035414000040...",8841
"""CS018415000234...",10008
"""CS029415000054...",9504
"""CS017415000187...",9948


## P-079
商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [114]:
df_product.null_count()

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
u32,u32,u32,u32,u32,u32
0,0,0,0,7,7


## P-080
商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。  
なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [115]:
# 元のレコード数
df_product.height

10030

In [116]:
# 欠損削除
df_product_drop_nulls = df_product.drop_nulls()

# レコード数確認
df_product_drop_nulls.height

10023

## P-081
単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。  
なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。  
補完実施後、各項目について欠損が生じていないことも確認すること。

In [117]:
# 平均が整数らしく丸めの必要が無かった
df_product.fill_null(strategy="mean").null_count()

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


## P-082
単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。  
なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。  
補完実施後、各項目について欠損が生じていないことも確認すること。

In [118]:
df_product.with_columns([
    pl.col("unit_price").fill_null(pl.col("unit_price").median()).round(0).cast(pl.UInt64),
    pl.col("unit_cost").fill_null(pl.col("unit_cost").median()).round(0).cast(pl.UInt64)
]).null_count()

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


## P-083
単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。  
なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。  
補完実施後、各項目について欠損が生じていないことも確認すること。

In [119]:
# caategory_small_cdごとに中央値を算出する、という
# Exprで欠損値を埋める必要があるので、柔軟なcoalesceを用いる
df_product.with_columns([
    pl.coalesce([
        pl.col("unit_price"),
        pl.col("unit_price").median().over("category_small_cd").cast(pl.UInt64)
    ]).alias("unit_price_fill_na"),
    pl.coalesce([
        pl.col("unit_cost"),
        pl.col("unit_cost").median().over("category_small_cd").cast(pl.UInt64)
    ]).alias("unit_cost_fill_na")
]).null_count()

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_fill_na,unit_cost_fill_na
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,7,7,0,0


## P-084
顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。  
ただし、売上実績がない場合は0として扱うこと。  
そして計算した割合が0超のものを抽出し、結果を10件表示せよ。  
また、作成したデータに欠損が存在しないことを確認せよ。

In [120]:
df_2019_ratio = df_customer.join(
    df_receipt, how="left", on="customer_id"
).with_columns(
    pl.col("amount").fill_null(0)  # 売上実績がない場合は0で埋める
).groupby("customer_id").agg([
    pl.sum("amount").alias("amount_sum_all"),
    pl.col("amount").filter(pl.col("sales_ymd").dt.year() == 2019).sum().alias("amount_sum_2019")
]).with_columns(
    (pl.col("amount_sum_2019").cast(pl.Float64) / pl.col("amount_sum_all").cast(pl.Float64)).alias("amount_2019_ratio")
)

df_2019_ratio.filter(
    pl.col("amount_2019_ratio") > 0
).select([
    "customer_id", "amount_sum_all", "amount_sum_2019",
    "amount_2019_ratio"
]).head(10)

customer_id,amount_sum_all,amount_sum_2019,amount_2019_ratio
str,u32,u32,f64
"""CS048212000004...",238,238,1.0
"""CS014414000069...",4162,874,0.209995
"""CS032414000003...",7976,2729,0.342151
"""CS021404000020...",804,476,0.59204
"""CS002415000515...",594,594,1.0
"""CS009311000020...",484,326,0.673554
"""CS023512000156...",208,208,1.0
"""CS010515000092...",2351,2029,0.863037
"""CS019414000004...",2194,992,0.452142


In [121]:
df_2019_ratio.filter(pl.col("amount_2019_ratio") > 0).null_count()

customer_id,amount_sum_all,amount_sum_2019,amount_2019_ratio
u32,u32,u32,u32
0,0,0,0


## P-085
顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。  
ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。  
また、作成結果を確認するために結果を10件表示せよ。

In [122]:
df_customer_geocode = df_customer.join(
    df_geocode.groupby("postal_cd").agg([
        pl.mean("longitude"),
        pl.mean("latitude")
    ]),
    how="inner", on="postal_cd"
)

df_customer_geocode.head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,longitude,latitude
str,str,u8,str,date,u8,str,str,str,date,str,f32,f32
"""CS012712000059...","""村井 綾女""",1,"""女性""",1945-01-02,74,"""231-0002""","""神奈川県横浜市中区海岸通**...","""S14012""",2018-07-20,"""0-00000000-0""",139.646835,35.450531
"""CS006514000017...","""柳田 涼""",1,"""女性""",1967-07-27,51,"""224-0056""","""神奈川県横浜市都筑区川和台*...","""S14006""",2015-10-06,"""7-20091229-9""",139.554443,35.52816
"""CS006715000029...","""大山 薫""",9,"""不明""",1940-02-25,79,"""224-0056""","""神奈川県横浜市都筑区川和台*...","""S14006""",2015-06-04,"""0-00000000-0""",139.554443,35.52816
"""CS006413000206...","""豊田 知世""",1,"""女性""",1974-06-01,44,"""224-0056""","""神奈川県横浜市都筑区川和台*...","""S14006""",2015-12-22,"""0-00000000-0""",139.554443,35.52816
"""CS006713000013...","""吉原 まさみ""",1,"""女性""",1941-05-16,77,"""224-0056""","""神奈川県横浜市都筑区川和台*...","""S14006""",2015-10-29,"""0-00000000-0""",139.554443,35.52816
"""CS006704000003...","""坂口 功補""",0,"""男性""",1947-02-20,72,"""224-0056""","""神奈川県横浜市都筑区川和台*...","""S14006""",2015-02-11,"""0-00000000-0""",139.554443,35.52816
"""CS006813000004...","""川井 愛梨""",1,"""女性""",1937-02-04,82,"""224-0056""","""神奈川県横浜市都筑区川和台*...","""S14006""",2015-02-02,"""0-00000000-0""",139.554443,35.52816
"""CS006803000004...","""窪田 明慶""",0,"""男性""",1933-06-17,85,"""224-0056""","""神奈川県横浜市都筑区川和台*...","""S14006""",2015-06-19,"""7-20090504-8""",139.554443,35.52816
"""CS006401000015...","""堤 人志""",0,"""男性""",1972-05-07,46,"""224-0056""","""神奈川県横浜市都筑区川和台*...","""S14006""",2015-09-27,"""0-00000000-0""",139.554443,35.52816


## P-086
085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。  
そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ  
。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。  
結果は10件表示せよ。

緯度（ラジアン）：ϕ  
経度（ラジアン）：λ  
距離 L = 6371 ∗ arccos(sinϕ1 ∗ sinϕ2 + cosϕ1 ∗ cosϕ2 ∗ cos(λ1 − λ2))

In [123]:
def calc_distance(
    lat1: str, lon1: str,
    lat2: str, lon2: str
) -> pl.Expr:
    
    import numpy as np
    
    lat1_rad = pl.col(lat1) * 180 / np.pi
    lat2_rad = pl.col(lat2) * 180 / np.pi
    lon1_rad = pl.col(lon1) * 180 / np.pi
    lon2_rad = pl.col(lon2) * 180 / np.pi
    
    L = 6371 * (
        lat1_rad.sin() * lat2_rad.sin() + lat1_rad.cos() * lat2_rad.cos() * (lon1_rad - lon2_rad).cos()
    ).arccos()
    
    return L

df_customer_geocode.join(
    df_store,
    how="inner",
    left_on="application_store_cd",
    right_on="store_cd",
    suffix="_store"
).select([
    "customer_id",
    "address",
    "address_store",
    calc_distance("latitude", "latitude_store", "longitude", "longitude_store").alias("distance")
]).head(10)

customer_id,address,address_store,distance
str,str,str,f32
"""CS012712000059...","""神奈川県横浜市中区海岸通**...","""神奈川県横浜市中区本牧和田""",7854.72168
"""CS006514000017...","""神奈川県横浜市都筑区川和台*...","""神奈川県横浜市都筑区葛が谷""",5047.092285
"""CS006715000029...","""神奈川県横浜市都筑区川和台*...","""神奈川県横浜市都筑区葛が谷""",5047.092285
"""CS006413000206...","""神奈川県横浜市都筑区川和台*...","""神奈川県横浜市都筑区葛が谷""",5047.092285
"""CS006713000013...","""神奈川県横浜市都筑区川和台*...","""神奈川県横浜市都筑区葛が谷""",5047.092285
"""CS006704000003...","""神奈川県横浜市都筑区川和台*...","""神奈川県横浜市都筑区葛が谷""",5047.092285
"""CS006813000004...","""神奈川県横浜市都筑区川和台*...","""神奈川県横浜市都筑区葛が谷""",5047.092285
"""CS006803000004...","""神奈川県横浜市都筑区川和台*...","""神奈川県横浜市都筑区葛が谷""",5047.092285
"""CS006401000015...","""神奈川県横浜市都筑区川和台*...","""神奈川県横浜市都筑区葛が谷""",5047.092285


## P-087
顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。  
名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。  
ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [124]:
df_customer_nayose = df_customer.join(
    df_receipt.groupby("customer_id").agg(pl.sum("amount")), how="left", on="customer_id"
).sort(
    ["amount", "customer_id"], reverse=[True, False]
).unique(subset=["customer_name", "postal_cd"], keep="first")

In [125]:
# データの件数
print(f"顧客データの件数: {df_customer.height}")
print(f"名寄顧客データの件数: {df_customer_nayose.height}")
print(f"重複数: {df_customer.height - df_customer_nayose.height}")

顧客データの件数: 21971
名寄顧客データの件数: 21941
重複数: 30


## P-088
087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。

- 重複していない顧客：顧客ID（customer_id）を設定
- 重複している顧客：前設問で抽出したレコードの顧客IDを設定

顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [126]:
df_customer_nayose_id = df_customer.join(
    df_customer_nayose.select([
        "customer_name", "postal_cd", pl.col("customer_id").alias("nayose_id")
    ]),
    how="inner", on=["customer_name", "postal_cd"]
)

df_customer_nayose_id.select([
    pl.col("customer_id").n_unique(),
    pl.col("nayose_id").n_unique()
])

customer_id,nayose_id
u32,u32
21971,21941


## P-089
売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [127]:
from sklearn.model_selection import train_test_split

df_model = df_receipt.groupby("customer_id").agg(
    pl.sum("amount")
).filter(pl.col("amount") > 0)

df_train, df_test = train_test_split(
    df_model, test_size=0.2, random_state=123
)

df_train.shape, df_test.shape, df_model.shape

((6645, 2), (1662, 2), (8307, 2))

## P-090
レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。  
売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [128]:
# polarsにはindexが無いので、pandasにしてからTimeSeriesSplitを用いる
import pandas as pd
import pyarrow
from sklearn.model_selection import TimeSeriesSplit

df = df_receipt.groupby([
    pl.col("sales_ymd").dt.year().alias("sales_year"),
    pl.col("sales_ymd").dt.month().alias("sales_month")
]).agg(
    pl.sum("amount")
).sort(["sales_year", "sales_month"]).to_pandas()

# TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3, max_train_size=12, test_size=6)

cvs = []
for train_idx, test_idx in tscv.split(df):
    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]
    cvs.append((train_df, test_df))
    
cvs[0]

(    sales_year  sales_month  amount
 4         2017            5  884010
 5         2017            6  894242
 6         2017            7  959205
 7         2017            8  954836
 8         2017            9  902037
 9         2017           10  905739
 10        2017           11  932157
 11        2017           12  939654
 12        2018            1  944509
 13        2018            2  864128
 14        2018            3  946588
 15        2018            4  937099,
     sales_year  sales_month   amount
 16        2018            5  1004438
 17        2018            6  1012329
 18        2018            7  1058472
 19        2018            8  1045793
 20        2018            9   977114
 21        2018           10  1069939)

## P-091
顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [134]:
# まず売り上げ実績がある顧客とない顧客を区別するフラグを作成する
df_sales = df_customer.join(
    df_receipt.groupby("customer_id").agg(
        pl.sum("amount")
    ).select([
        "customer_id", "amount"
    ]),
    how="left",
    on="customer_id"
).select([
    "customer_id",
    pl.when(pl.col("amount").is_null()).then(0)
      .otherwise(1).alias("sales_flg")
])

# フラグごとの顧客数を確認
df_sales.select(pl.col("sales_flg").value_counts())

sales_flg
struct[2]
"{0,13665}"
"{1,8306}"


In [135]:
# imblearnでアンダーサンプリングする
from imblearn.under_sampling import RandomUnderSampler

# Pandasに変換
df_sales = df_sales.to_pandas()

# アンダーサンプリング
X = df_sales[["customer_id"]]
y = df_sales["sales_flg"]

rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

y_resampled.value_counts()

0    8306
1    8306
Name: sales_flg, dtype: int64

## P-092
顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

In [138]:
# 性別マスタ
df_gender_std3 = df_customer.select(["gender_cd", "gender"]).unique()

# 性別を分離
df_customer_gender_std3 = df_customer.drop("gender")

In [139]:
df_gender_std3

gender_cd,gender
u8,str
1,"""女性"""
9,"""不明"""
0,"""男性"""


In [140]:
df_customer_gender_std3.head(3)

customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,u8,date,u8,str,str,str,date,str
"""CS021313000114...","""大野 あや子""",1,1981-04-29,37,"""259-1113""","""神奈川県伊勢原市粟窪****...","""S14021""",2015-09-05,"""0-00000000-0"""
"""CS037613000071...","""六角 雅彦""",9,1952-04-01,66,"""136-0076""","""東京都江東区南砂******...","""S13037""",2015-04-14,"""0-00000000-0"""
"""CS031415000172...","""宇多田 貴美子""",1,1976-10-04,42,"""151-0053""","""東京都渋谷区代々木*****...","""S13031""",2015-05-29,"""D-20100325-C"""


## P-093
商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。  
カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [145]:
df_093 = df_product.join(
    df_category.select(["category_major_name", "category_medium_name", "category_small_name", "category_small_cd"])
    , how="inner"
    , on="category_small_cd"
)

df_093.head(3)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
str,str,str,str,u64,u64,str,str,str
"""P040101001""","""04""","""0401""","""040101""",198,149,"""惣菜""","""御飯類""","""弁当類"""
"""P040101002""","""04""","""0401""","""040101""",218,164,"""惣菜""","""御飯類""","""弁当類"""
"""P040101003""","""04""","""0401""","""040101""",230,173,"""惣菜""","""御飯類""","""弁当類"""


## P-094
093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。

| フィル形式 | ヘッダ有無 | エンコーディング | 
| ---- | ---- | ---- |
| csv | 有り | UTF-8 |

ファイル出力先のパスは `./data` とすること。

In [146]:
df_093.write_csv("./data/df_093.csv")

## P-095
093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。

| フィル形式 | ヘッダ有無 | エンコーディング | 
| ---- | ---- | ---- |
| csv | 有り | CP932 |

ファイル出力先のパスは `./data` とすること。

In [147]:
# PolarsではCP932でエンコーディングできないのでPandasに変換
df_093_pd = df_093.to_pandas()
df_093_pd.to_csv("./data/df_093_cp932.csv", encoding="CP932", index=False)

## P-096
093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。

| フィル形式 | ヘッダ有無 | エンコーディング | 
| ---- | ---- | ---- |
| csv | 無し | UTF-8 |

ファイル出力先のパスは `./data` とすること。

In [149]:
df_093.write_csv("./data/df_093_without_header.csv", has_header=False)

## P-097
094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。

In [151]:
pl.read_csv("./data/df_093.csv").head(3)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
str,i64,i64,i64,i64,i64,str,str,str
"""P040101001""",4,401,40101,198,149,"""惣菜""","""御飯類""","""弁当類"""
"""P040101002""",4,401,40101,218,164,"""惣菜""","""御飯類""","""弁当類"""
"""P040101003""",4,401,40101,230,173,"""惣菜""","""御飯類""","""弁当類"""


## P-098
096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。

In [152]:
pl.read_csv("./data/df_093_without_header.csv").head(3)

P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
str,i64,i64,i64,i64,i64,str,str,str
"""P040101002""",4,401,40101,218,164,"""惣菜""","""御飯類""","""弁当類"""
"""P040101003""",4,401,40101,230,173,"""惣菜""","""御飯類""","""弁当類"""
"""P040101004""",4,401,40101,248,186,"""惣菜""","""御飯類""","""弁当類"""


## P-099
093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。

| フィル形式 | ヘッダ有無 | エンコーディング | 
| ---- | ---- | ---- |
| tsv | 有り | UTF-8 |

ファイル出力先のパスは `./data` とすること。

In [154]:
df_093.write_csv("./data/df_093.tsv", sep="\t")

## P-100
099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。

In [155]:
pl.read_csv("./data/df_093.tsv", sep="\t").head(3)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
str,i64,i64,i64,i64,i64,str,str,str
"""P040101001""",4,401,40101,198,149,"""惣菜""","""御飯類""","""弁当類"""
"""P040101002""",4,401,40101,218,164,"""惣菜""","""御飯類""","""弁当類"""
"""P040101003""",4,401,40101,230,173,"""惣菜""","""御飯類""","""弁当類"""
